In [1]:
import requests
import json
from typing import Any, Text, Dict, List
from rasa_sdk import Action, Tracker
from rasa_sdk.executor import CollectingDispatcher
from rasa_sdk.events import SlotSet
from SPARQLWrapper import SPARQLWrapper, JSON
from datetime import datetime
import logging
from decimal import Decimal
import mysql.connector
from mysql.connector import Error
import pandas as pd
import logging
import numpy as np

In [36]:
# def t5_sparql_to_sql(Nl, device_id, sparql_output):
#     """
#     Simulate converting a SPARQL output to an SQL query.
#     For this example, we assume the SPARQL output contains device_id = 1.
#     We generate an SQL query to select all sensor data for device_id 1 for the day 2025-03-17.
#     """
#     sql_query = """
#     SELECT *
#     FROM sensor_data
#     WHERE device_id = 1
#       AND `Datetime` BETWEEN '2025-03-17 00:00:00' AND '2025-03-17 23:59:59'
#     ORDER BY `Datetime` ASC;
#     """
#     return sql_query

# # Simulate SPARQL output from T5 model
# sparql_output = {"hasTimeseriesId": "1", "sensor_id" = "Air_Temperature_Sensor", "startDate": '2025-02-10 00:00:00', "endDate": '2025-02-20 23:59:59'}
# sql_query = t5_sparql_to_sql(sparql_output)
# print("Generated SQL Query:")
# print(sql_query)

SyntaxError: ':' expected after dictionary key (1228648190.py, line 17)

In [4]:
sql_query = """ \
"SELECT Air_Temperature_Sensor
FROM sensor_data
WHERE device_id = 1
AND `Datetime` BETWEEN '2025-02-10 00:00:00' AND '2025-02-20 23:59:59';" \
"""

In [3]:
import mysql.connector
from mysql.connector import Error
import json
import logging
from decimal import Decimal

# Assume DB_CONFIG is defined globally:
DB_CONFIG = {
    "host": "localhost",
    "database": "sensordb",
    "user": "root",
    "password": "root",
}

In [ ]:
def fetch_sql_data(
    timeseriesId, start_date, end_date, database, table_name, return_json=True
):
    """
    Fetches sensor data for multiple timeseries IDs and dates dynamically.

    Parameters:
      timeseriesId: will be single or multiple UUIDs such as `249a4c9c-fe31-4649-a119-452e5e8e7dc5`
      start_time: Start timestamp as a string, e.g., '2025-02-10 00:00:00'.
      end_time: End timestamp as a string, e.g., '2025-02-20 23:59:59'.
      database and table_name: will also be a variable placeholder and database name as well . user will add this form outsite the function
      return_json: If True, returns the results as a JSON string; otherwise, returns a Python dict.

    Returns:
      A tuple (results, error) where error is None if successful.

    Expected Output Example (not well formatted):
      {
              "timeseriesId 1": {
                  "[
                      {"datetime": "2025-02-10 05:31:59", "reading_value": 27.99},
                      ...
                  ]
              },
         "timeseriesId 2": 
          {
              "timeseriesId 3": {
                [
                      {"datetime": "2025-02-10 06:00:00", "reading_value": 28.10},
                      ...
                  ]
              },
              "timeseriesId 4": {
           [
                      {"datetime": "2025-02-10 06:05:00", "reading_value": 29.00},
                      ...
                  ]
              }
          }
      }
    """

    try:
        connection = mysql.connector.connect(**DB_CONFIG)
        if connection.is_connected():
            cursor = connection.cursor(dictionary=True)
            results = {}

            for timeseries_id, sensors in timeseries_sensors_map.items():
                results[str(timeseries_id)] = {}

                for sensor_name in sensors:
                    query = f"""
                    
                    """
                    cursor.execute(query, (timeseries_id, start_time, end_time))
                    rows = cursor.fetchall()

                    # Initialize sensor data
                    sensor_data = {"timeseries_data": []}

                    # Fetch data for the current sensor
                    for row in rows:
                        dt_value = row.get("Datetime")
                        reading_value = row.get("reading_value")

                        # Convert reading_value from Decimal to float if necessary
                        if isinstance(reading_value, Decimal):
                            reading_value = float(reading_value)

                        # Convert datetime to string if needed
                        if dt_value is not None and hasattr(dt_value, "strftime"):
                            dt_value = dt_value.strftime("%Y-%m-%d %H:%M:%S")

                        # Append reading to timeseries_data
                        sensor_data["timeseries_data"].append(
                            {"datetime": dt_value, "reading_value": reading_value}
                        )

                    # Add sensor_data to results for the current timeseriesId
                    results[str(timeseries_id)][sensor_name] = sensor_data

            cursor.close()
            connection.close()

            if return_json:
                return json.dumps(results, indent=4), None
            else:
                return results, None

    except Error as e:
        logging.error(f"MySQL error: {e}")
        return None, str(e)

In [4]:
import json
import logging
from typing import List, Tuple, Union, Dict
import mysql.connector
from decimal import Decimal

def fetch_sql_data(
    timeseries_ids: Union[str, List[str]],
    start_date: str,
    end_date: str,
    database: str,
    table_name: str,
    db_config: Dict,
    return_json: bool = True
) -> Tuple[Union[str, Dict], Union[str, None]]:
    """
    Fetches sensor data for multiple timeseries IDs and dates dynamically.

    Parameters:
        timeseries_ids: Single UUID or list of UUIDs (e.g., '249a4c9c-fe31-4649-a119-452e5e8e7dc5').
        start_date: Start timestamp as a string, e.g., '2025-02-10 00:00:00'.
        end_date: End timestamp as a string, e.g., '2025-02-20 23:59:59'.
        database: Name of the database (e.g., 'sensordb').
        table_name: Name of the table (e.g., 'sensor_data').
        db_config: Dictionary containing database connection parameters (host, user, password, etc.).
        return_json: If True, returns results as a JSON string; otherwise, returns a Python dict.

    Returns:
        A tuple (results, error) where error is None if successful. Results are formatted as:
        {
            "timeseriesId_1": [
                {"datetime": "2025-02-10 05:31:59", "reading_value": 27.99},
                ...
            ],
            "timeseriesId_2": [
                {"datetime": "2025-02-10 06:00:00", "reading_value": 28.10},
                ...
            ],
            ...
        }
    """
    try:
        # Ensure timeseries_ids is a list
        if isinstance(timeseries_ids, str):
            timeseries_ids = [timeseries_ids]

        # Validate inputs
        if not timeseries_ids or not all(isinstance(tid, str) for tid in timeseries_ids):
            return None, "Invalid or empty timeseries_ids provided"
        if not start_date or not end_date:
            return None, "Start_date and end_date must be provided"
        if not database or not table_name:
            return None, "Database and table_name must be provided"

        # Establish database connection
        connection = mysql.connector.connect(**db_config)
        if not connection.is_connected():
            return None, "Failed to connect to the database"

        cursor = connection.cursor(dictionary=True)
        results = {}

        # Construct dynamic SQL query
        # Select Datetime and all timeseries IDs as columns
        columns = ["Datetime"] + [f"`{tid}`" for tid in timeseries_ids]
        columns_str = ", ".join(columns)

        # Construct WHERE clause to ensure non-NULL values for all timeseries IDs
        where_conditions = [f"`{tid}` IS NOT NULL" for tid in timeseries_ids]
        where_conditions.append("Datetime BETWEEN %s AND %s")
        where_clause = " AND ".join(where_conditions)

        # Build the full query
        query = f"""
            SELECT {columns_str}
            FROM `{database}`.`{table_name}`
            WHERE {where_clause}
        """

        # Execute query with parameters
        cursor.execute(query, (start_date, end_date))
        rows = cursor.fetchall()

        # Initialize results dictionary
        for tid in timeseries_ids:
            results[tid] = []

        # Process each row
        for row in rows:
            dt_value = row.get("Datetime")
            if dt_value and hasattr(dt_value, "strftime"):
                dt_value = dt_value.strftime("%Y-%m-%d %H:%M:%S")

            # Add data for each timeseries ID
            for tid in timeseries_ids:
                reading_value = row.get(tid)
                if reading_value is not None:
                    # Convert Decimal to float if necessary
                    if isinstance(reading_value, Decimal):
                        reading_value = float(reading_value)
                    results[tid].append(
                        {"datetime": dt_value, "reading_value": reading_value}
                    )

        # Clean up
        cursor.close()
        connection.close()

        # Return results in the requested format
        if return_json:
            return json.dumps(results, indent=4), None
        return results, None

    except mysql.connector.Error as e:
        logging.error(f"MySQL error: {e}")
        return None, f"MySQL error: {str(e)}"
    except Exception as e:
        logging.error(f"Unexpected error: {e}")
        return None, f"Unexpected error: {str(e)}"

In [5]:
import logging
from datetime import datetime

# Database configuration
DB_CONFIG = {
    "host": "localhost",
    "database": "sensordb",
    "user": "root",
    "password": "root",
}

# Timeseries IDs
timeseries_ids = [
    "0861df67-0088-4143-a14d-8c371d243121",
    "95ae1ca6-1806-40b9-9493-9b6be51a5e03"
]

# Date range
start_date = "2025-02-15 00:00:00"
end_date = "2025-02-16 23:59:59"

# Database and table name
database = "sensordb"
table_name = "sensor_data"


json_results = fetch_sql_data(
        timeseries_ids=timeseries_ids,
        start_date=start_date,
        end_date=end_date,
        database=database,
        table_name=table_name,
        db_config=DB_CONFIG,
        return_json=True
    )
print("JSON Results:")
print(json_results)

JSON Results:
('{\n    "0861df67-0088-4143-a14d-8c371d243121": [\n        {\n            "datetime": "2025-02-15 00:02:01",\n            "reading_value": 10\n        },\n        {\n            "datetime": "2025-02-15 00:11:34",\n            "reading_value": 10\n        },\n        {\n            "datetime": "2025-02-15 00:12:20",\n            "reading_value": 10\n        },\n        {\n            "datetime": "2025-02-15 00:17:30",\n            "reading_value": 10\n        },\n        {\n            "datetime": "2025-02-15 00:17:41",\n            "reading_value": 10\n        },\n        {\n            "datetime": "2025-02-15 00:19:01",\n            "reading_value": 11\n        },\n        {\n            "datetime": "2025-02-15 00:20:45",\n            "reading_value": 10\n        },\n        {\n            "datetime": "2025-02-15 00:23:13",\n            "reading_value": 10\n        },\n        {\n            "datetime": "2025-02-15 00:31:48",\n            "reading_value": 10\n        },

In [ ]:
def fetch_sensor_data(
    timeseries_sensors_map, start_time, end_time, table_name, return_json=True
):
    """
    Fetches sensor data for multiple timeseries IDs and multiple sensors dynamically.

    Parameters:
      timeseries_sensors_map: A dictionary where keys are timeseriesIds and values are lists of sensor names.
                              Example: {1: ["Air_Temperature_Sensor", "Zone_Air_Humidity_Sensor"],
                                        2: ["Air_Temperature_Sensor", "Zone_Air_Humidity_Sensor"]}
      start_time: Start timestamp as a string, e.g., '2025-02-10 00:00:00'.
      end_time: End timestamp as a string, e.g., '2025-02-20 23:59:59'.
      table_name: Name of the table, e.g., "sensor_data".
      return_json: If True, returns the results as a JSON string; otherwise, returns a Python dict.

    Returns:
      A tuple (results, error) where error is None if successful.

    Output Example:
      {
          "1": {
              "Air_Temperature_Sensor_x": {
                  "timeseries_data": [
                      {"datetime": "2025-02-10 05:31:59", "reading_value": 27.99},
                      ...
                  ]
              },
              "Zone_Air_Humidity_Sensor_y": {
                  "timeseries_data": [
                      {"datetime": "2025-02-10 05:35:00", "reading_value": 28.05},
                      ...
                  ]
              }
          },
          "2": {
              "Air_Temperature_Sensor": {
                  "timeseries_data": [
                      {"datetime": "2025-02-10 06:00:00", "reading_value": 28.10},
                      ...
                  ]
              },
              "Zone_Air_Humidity_Sensor": {
                  "timeseries_data": [
                      {"datetime": "2025-02-10 06:05:00", "reading_value": 29.00},
                      ...
                  ]
              }
          }
      }
    """

    try:
        connection = mysql.connector.connect(**DB_CONFIG)
        if connection.is_connected():
            cursor = connection.cursor(dictionary=True)
            results = {}

            for timeseries_id, sensors in timeseries_sensors_map.items():
                results[str(timeseries_id)] = {}

                for sensor_name in sensors:
                    query = f"""
                    SELECT `Datetime`, {sensor_name} AS reading_value, timeseriesId
                    FROM {table_name}
                    WHERE timeseriesId = %s
                      AND `Datetime` BETWEEN %s AND %s
                    ORDER BY `Datetime` ASC;
                    """
                    cursor.execute(query, (timeseries_id, start_time, end_time))
                    rows = cursor.fetchall()

                    # Initialize sensor data
                    sensor_data = {"timeseries_data": []}

                    # Fetch data for the current sensor
                    for row in rows:
                        dt_value = row.get("Datetime")
                        reading_value = row.get("reading_value")

                        # Convert reading_value from Decimal to float if necessary
                        if isinstance(reading_value, Decimal):
                            reading_value = float(reading_value)

                        # Convert datetime to string if needed
                        if dt_value is not None and hasattr(dt_value, "strftime"):
                            dt_value = dt_value.strftime("%Y-%m-%d %H:%M:%S")

                        # Append reading to timeseries_data
                        sensor_data["timeseries_data"].append(
                            {"datetime": dt_value, "reading_value": reading_value}
                        )

                    # Add sensor_data to results for the current timeseriesId
                    results[str(timeseries_id)][sensor_name] = sensor_data

            cursor.close()
            connection.close()

            if return_json:
                return json.dumps(results, indent=4), None
            else:
                return results, None

    except Error as e:
        logging.error(f"MySQL error: {e}")
        return None, str(e)

In [ ]:
if __name__ == "__main__":
    # Example usage to fetch data for multiple sensors and timeseries IDs

    # Example parameters
    timeseries_sensors_map = {
        2: [
            "Air_Temperature_Sensor",
            "Zone_Air_Humidity_Sensor",
            "PM2_5_Level_Sensor_Standard",
            # "PM10_Level_Sensor_Standard",
            # "NO2_Level_Sensor",
            # "CO_Level_Sensor",
            # "CO2_Level_Sensor",
            # "Sound_Noise_Sensor_MEMS",
            # "Air_Quality_Sensor",
            # "Formaldehyde_Level_Sensor",
            # "PM1_Level_Sensor_Standard",
        ]
        # 2: ["Air_Temperature_Sensor", "Zone_Air_Humidity_Sensor"],
    }
    start_time = "2025-02-10 00:00:00"
    end_time = "2025-02-10 01:00:00"
    table_name = "sensor_data"

    # Fetch data for multiple sensors and timeseries IDs
    json_data, error = fetch_sensor_data(
        timeseries_sensors_map, start_time, end_time, table_name
    )

    if error:
        print(f"Error fetching data: {error}")
    else:
        print("Sensor Data:")
        print(json_data)

In [ ]:
def analyze_recalibration_frequency(sensor_data):
    """
    Analyzes recalibration frequency for sensors given grouped sensor data with a nested structure.

    Expected input format (as a Python dict or JSON string):
      {
          "1": {
              "Air_Temperature_Sensor": {
                  "timeseries_data": [
                      {"datetime": "2025-02-10 05:31:59", "reading_value": 27.99},
                      ...
                  ]
              },
              "Zone_Air_Humidity_Sensor": {
                  "timeseries_data": [
                      {"datetime": "2025-02-10 05:35:00", "reading_value": 28.05},
                      ...
                  ]
              }
          },
          "2": {
              "Air_Temperature_Sensor": {
                  "timeseries_data": [
                      {"datetime": "2025-02-10 06:00:00", "reading_value": 28.10},
                      ...
                  ]
              },
              "Zone_Air_Humidity_Sensor": {
                  "timeseries_data": [
                      {"datetime": "2025-02-10 06:05:00", "reading_value": 29.00},
                      ...
                  ]
              }
          }
      }

    For each sensor type, it calculates the mean and standard deviation of the reading_value,
    computes the coefficient of variation (cv = std / mean), and then:
      - If cv > 0.1, indicates high variability (suggesting more frequent recalibration).
      - Otherwise, indicates stable performance.

    Returns:
      A nested dictionary where each sensor ID maps to sensor type keys and their analysis results.
    """
    # If sensor_data is a JSON string, convert it into a dictionary.
    if isinstance(sensor_data, str):
        try:
            sensor_data = json.loads(sensor_data)
        except Exception as e:
            logging.error(f"Error parsing sensor_data JSON: {e}")
            return {"error": "Invalid sensor_data JSON"}

    response = {}
    # Iterate over each sensor ID.
    for sensor_id, sensor_types in sensor_data.items():
        response[sensor_id] = {}
        # Iterate over each sensor type for this sensor ID.
        for sensor_type, sensor_info in sensor_types.items():
            # Get the list of readings from the "timeseries_data" key.
            timeseries_data = sensor_info.get("timeseries_data", [])
            try:
                df = pd.DataFrame(timeseries_data)
                # Rename "datetime" column to "timestamp" if it exists.
                if "datetime" in df.columns:
                    df = df.rename(columns={"datetime": "timestamp"})
                # Convert the timestamp column to datetime objects.
                df["timestamp"] = pd.to_datetime(df["timestamp"])
            except Exception as e:
                logging.error(
                    f"Data conversion error for sensor {sensor_id}, type {sensor_type}: {e}"
                )
                response[sensor_id][sensor_type] = {
                    "error": "Invalid sensor data format"
                }
                continue

            # Compute mean and standard deviation of the reading values.
            mean_val = df["reading_value"].mean()
            std_val = df["reading_value"].std() or 0.0
            cv = std_val / mean_val if mean_val else 0

            if cv > 0.1:
                response[sensor_id][sensor_type] = {
                    "mean": mean_val,
                    "std": std_val,
                    "coefficient_of_variation": cv,
                    "message": f"Sensor {sensor_id} ({sensor_type}) has high variability; recalibration might be required more frequently.",
                }
            else:
                response[sensor_id][sensor_type] = {
                    "mean": mean_val,
                    "std": std_val,
                    "coefficient_of_variation": cv,
                    "message": "Sensor performance is stable; no immediate recalibration needed.",
                }
    if not response:
        return {"message": "No sensor data available."}

    return response

In [48]:
sample_json = json_data
result = analyze_recalibration_frequency(sample_json)
print(json.dumps(result, indent=2))

{
  "2": {
    "Air_Temperature_Sensor": {
      "mean": 28.98535771065183,
      "std": 0.03853289840796687,
      "coefficient_of_variation": 0.00132939185338418,
      "message": "Sensor performance is stable; no immediate recalibration needed."
    }
  }
}


In [49]:
import json
import logging
import pandas as pd


def analyze_failure_trends(sensor_data):
    """
    Analyzes failure trends from grouped sensor data with a nested structure.

    Expected input format:
      {
          "1": {
              "Air_Temperature_Sensor": {
                  "timeseries_data": [
                      {"datetime": "2025-02-10 05:31:59", "reading_value": 27.99},
                      ...
                  ]
              },
              "Zone_Air_Humidity_Sensor": {
                  "timeseries_data": [
                      {"datetime": "2025-02-10 05:35:00", "reading_value": 28.05},
                      ...
                  ]
              }
          },
          "2": {
              "Air_Temperature_Sensor": {
                  "timeseries_data": [
                      {"datetime": "2025-02-10 06:00:00", "reading_value": 28.10},
                      ...
                  ]
              },
              "Zone_Air_Humidity_Sensor": {
                  "timeseries_data": [
                      {"datetime": "2025-02-10 06:05:00", "reading_value": 29.00},
                      ...
                  ]
              }
          }
      }

    For each sensor type:
      - Converts the list of readings into a DataFrame.
      - Renames the "datetime" column to "timestamp" and converts it to datetime objects.
      - Filters for readings from the last 24 hours.
      - Computes a rolling average and rolling standard deviation (window of 5).
      - Computes the overall (baseline) standard deviation.
      - Compares the latest rolling standard deviation against 1.5× the baseline.
      - Flags the sensor if the latest rolling std exceeds that threshold.

    Returns:
      A nested dictionary where each sensor ID maps to sensor type keys with their analysis summary.
    """
    # Parse JSON string input if needed
    if isinstance(sensor_data, str):
        try:
            sensor_data = json.loads(sensor_data)
        except Exception as e:
            logging.error(f"Error parsing sensor_data JSON: {e}")
            return {"error": "Invalid sensor_data JSON"}

    response = {}
    now = pd.Timestamp.now()

    # Iterate over each sensor ID
    for sensor_id, sensor_types in sensor_data.items():
        response[sensor_id] = {}
        # Iterate over each sensor type for the given sensor ID
        for sensor_type, sensor_info in sensor_types.items():
            # Get the list of readings from the "timeseries_data" key
            timeseries_data = sensor_info.get("timeseries_data", [])
            try:
                df = pd.DataFrame(timeseries_data)
                # Rename "datetime" column to "timestamp" if it exists
                if "datetime" in df.columns:
                    df = df.rename(columns={"datetime": "timestamp"})
                df["timestamp"] = pd.to_datetime(df["timestamp"])
            except Exception as e:
                logging.error(
                    f"Data conversion error for sensor {sensor_id}, type {sensor_type}: {e}"
                )
                response[sensor_id][sensor_type] = {
                    "error": "Invalid sensor data format"
                }
                continue

            # Filter for readings in the last 24 hours
            sensor_df = df[df["timestamp"] >= now - pd.Timedelta(hours=24)]
            if sensor_df.empty:
                response[sensor_id][sensor_type] = {
                    "message": "No recent data available."
                }
                continue

            # Sort by timestamp and compute rolling statistics
            sensor_df = sensor_df.sort_values(by="timestamp")
            sensor_df["rolling_avg"] = (
                sensor_df["reading_value"].rolling(window=5, min_periods=1).mean()
            )
            sensor_df["rolling_std"] = (
                sensor_df["reading_value"].rolling(window=5, min_periods=1).std()
            )

            # Compute baseline standard deviation and latest rolling standard deviation
            baseline_std = sensor_df["reading_value"].std() or 0.0
            current_std = sensor_df["rolling_std"].iloc[-1] or 0.0

            # Compare current rolling std with 1.5 times the baseline
            if baseline_std > 0 and current_std > 1.5 * baseline_std:
                response[sensor_id][sensor_type] = {
                    "historical_mean": sensor_df["reading_value"].mean(),
                    "historical_std": baseline_std,
                    "latest_rolling_std": current_std,
                    "message": f"Sensor {sensor_id} ({sensor_type}) shows increased variance suggesting potential failure.",
                }
            else:
                response[sensor_id][sensor_type] = {
                    "historical_mean": sensor_df["reading_value"].mean(),
                    "historical_std": baseline_std,
                    "latest_rolling_std": current_std,
                    "message": f"Sensor {sensor_id} ({sensor_type}) readings are within normal range.",
                }
    if not response:
        return {
            "message": "No sensor data available for analysis in the last 24 hours."
        }

    return response

In [51]:
sample_json = json_data
result = analyze_failure_trends(sample_json)
print(json.dumps(result, indent=2))

{
  "2": {
    "Air_Temperature_Sensor": {
      "message": "No recent data available."
    }
  }
}


In [22]:
def analyze_device_deviation(sensor_data):
    """
    Analyzes deviation for each sensor in nested JSON structure.

    Input format:
    {
        "1": {
            "Air_Temperature_Sensor": {
                "timeseries_data": [
                    {"datetime": "2025-02-10 05:31:59", "reading_value": 27.99},
                    ...
                ]
            },
            ...
        },
        ...
    }

    For each sensor, it:
      - Calculates historical mean, std deviation.
      - Finds latest reading.
      - Flags if latest reading deviates beyond 2 standard deviations.

    Returns:
        A nested dict with analysis results.
    """
    if isinstance(sensor_data, str):
        try:
            sensor_data = json.loads(sensor_data)
        except Exception as e:
            logging.error(f"Error parsing JSON: {e}")
            return {"error": "Invalid JSON"}

    response = {}

    for sensor_id, sensors in sensor_data.items():
        response[sensor_id] = {}

        for sensor_name, sensor_content in sensors.items():
            readings = sensor_content.get("timeseries_data", [])
            if not readings:
                response[sensor_id][sensor_name] = {"message": "No data available"}
                continue

            try:
                df = pd.DataFrame(readings)
                if "datetime" in df.columns:
                    df.rename(columns={"datetime": "timestamp"}, inplace=True)
                df["timestamp"] = pd.to_datetime(df["timestamp"])
                df.sort_values("timestamp", inplace=True)

                historical_mean = float(df["reading_value"].mean())
                historical_std = float(df["reading_value"].std() or 0.0)
                latest_reading = float(df.iloc[-1]["reading_value"])

                deviation_flag = False
                if historical_std > 0 and (
                    latest_reading < historical_mean - 2 * historical_std
                    or latest_reading > historical_mean + 2 * historical_std
                ):
                    deviation_flag = True

                response[sensor_id][sensor_name] = {
                    "historical_mean": round(historical_mean, 4),
                    "historical_std": round(historical_std, 4),
                    "latest_reading": round(latest_reading, 4),
                    "message": (
                        f"Deviation detected beyond 2 STD."
                        if deviation_flag
                        else "Within normal range."
                    ),
                }

            except Exception as e:
                logging.error(
                    f"Processing error for sensor {sensor_id} ({sensor_name}): {e}"
                )
                response[sensor_id][sensor_name] = {"error": "Processing failed"}

    return response

In [23]:
sample_json = json_data
result = analyze_device_deviation(sample_json)
print(json.dumps(result, indent=2))

{
  "2": {
    "Air_Temperature_Sensor": {
      "historical_mean": 29.014,
      "historical_std": 0.025,
      "latest_reading": 28.97,
      "message": "Within normal range."
    },
    "Zone_Air_Humidity_Sensor": {
      "historical_mean": 23.2919,
      "historical_std": 0.075,
      "latest_reading": 23.2,
      "message": "Within normal range."
    },
    "PM2_5_Level_Sensor_Standard": {
      "historical_mean": 5.8127,
      "historical_std": 0.7486,
      "latest_reading": 6.0,
      "message": "Within normal range."
    },
    "PM10_Level_Sensor_Standard": {
      "historical_mean": 5.8127,
      "historical_std": 0.7486,
      "latest_reading": 6.0,
      "message": "Within normal range."
    },
    "NO2_Level_Sensor": {
      "historical_mean": 17.5238,
      "historical_std": 0.5002,
      "latest_reading": 17.0,
      "message": "Within normal range."
    },
    "CO_Level_Sensor": {
      "historical_mean": 137.4698,
      "historical_std": 0.7193,
      "latest_reading":

In [ ]:
def analyze_sensor_status(sensor_data):
    """
    Analyzes the status of sensors based on their latest reporting timestamp.

    Expected input (nested format):
      {
          "1": {
              "Air_Temperature_Sensor": {
                  "timeseries_data": [
                      {"datetime": "2025-02-10 05:31:59", "reading_value": 27.99},
                      {"datetime": "2025-02-10 05:32:11", "reading_value": 27.99},
                      ...
                  ]
              },
              "Zone_Air_Humidity_Sensor": {
                  "timeseries_data": [
                      {"datetime": "2025-02-10 05:35:00", "reading_value": 28.05},
                      {"datetime": "2025-02-10 05:35:12", "reading_value": 28.07},
                      ...
                  ]
              }
          },
          "2": {
              ...
          }
      }

    For each sensor type:
      - Converts the "timeseries_data" into a DataFrame.
      - Renames the "datetime" column to "timestamp" (if necessary) and converts it to datetime objects.
      - Finds the latest timestamp.
      - If the latest report is older than 1 hour from now, marks the sensor as "offline"; otherwise "online".

    Returns:
      A nested dictionary where each sensor ID maps to sensor type keys with their analysis, including:
        - last_report: the most recent report time as a string.
        - status: "online" or "offline".
        - message: a descriptive message.
    """
    # If sensor_data is a JSON string, convert it to a dictionary.
    if isinstance(sensor_data, str):
        try:
            sensor_data = json.loads(sensor_data)
        except Exception as e:
            logging.error(f"Error parsing sensor_data JSON: {e}")
            return {"error": "Invalid sensor_data JSON"}

    # If sensor_data is a list, assume it's data for a single sensor and wrap it in a dictionary.
    if isinstance(sensor_data, list):
        sensor_data = {"1": sensor_data}

    response = {}
    now = pd.Timestamp.now()
    threshold = now - pd.Timedelta(hours=1)

    # Process data for each sensor ID.
    for sensor_id, sensor_types in sensor_data.items():
        response[sensor_id] = {}
        # Process each sensor type for the current sensor ID.
        for sensor_type, sensor_info in sensor_types.items():
            # Extract the list of readings from the "timeseries_data" key.
            timeseries_data = sensor_info.get("timeseries_data", [])
            try:
                df = pd.DataFrame(timeseries_data)
                # Rename "datetime" to "timestamp" if it exists.
                if "datetime" in df.columns:
                    df = df.rename(columns={"datetime": "timestamp"})
                df["timestamp"] = pd.to_datetime(df["timestamp"])
            except Exception as e:
                logging.error(
                    f"Data conversion error for sensor {sensor_id}, type {sensor_type}: {e}"
                )
                response[sensor_id][sensor_type] = {
                    "error": "Invalid sensor data format"
                }
                continue

            # Find the most recent timestamp for the sensor type.
            last_report = df["timestamp"].max()
            if pd.isna(last_report) or last_report < threshold:
                status = "offline"
                message = f"Sensor {sensor_id} ({sensor_type}) appears offline or not reporting recently."
            else:
                status = "online"
                message = f"Sensor {sensor_id} ({sensor_type}) is reporting data normally. Last report at {last_report.strftime('%Y-%m-%d %H:%M:%S')}."

            response[sensor_id][sensor_type] = {
                "last_report": (
                    last_report.strftime("%Y-%m-%d %H:%M:%S")
                    if not pd.isna(last_report)
                    else None
                ),
                "status": status,
                "message": message,
            }

    if not response:
        return {"message": "No sensor data available for analysis."}

    return response

In [58]:
sample_json = json_data
result = analyze_sensor_status(sample_json)
print(json.dumps(result, indent=2))

{
  "2": {
    "Air_Temperature_Sensor": {
      "last_report": "2025-02-10 01:59:49",
      "status": "offline",
      "message": "Sensor 2 (Air_Temperature_Sensor) appears offline or not reporting recently."
    }
  }
}


In [32]:
import json
import logging
import pandas as pd


def analyze_air_quality_trends(sensor_data, target_sensor="Air_Quality_Sensor"):
    """
    Analyzes air quality (or specified target sensor) trends from nested sensor data.

    Expected input (as Python dict or JSON string):
      {
          "1": {
              "Air_Quality_Sensor": {
                  "timeseries_data": [
                      {"datetime": "2025-02-10 05:31:59", "reading_value": 80},
                      {"datetime": "2025-02-10 05:32:11", "reading_value": 78},
                      ...
                  ]
              },
              ...
          },
          "2": {
              "Air_Quality_Sensor": {
                  "timeseries_data": [
                      {"datetime": "2025-02-10 06:00:00", "reading_value": 85},
                      ...
                  ]
              },
              ...
          }
      }

    For each sensor ID and the target sensor:
      - Computes mean (norm), latest reading, and determines trend: "rising", "falling", or "stable".

    Parameters:
        sensor_data: JSON or dict
        target_sensor: the sensor to analyze trends for (default: Air_Quality_Sensor)

    Returns:
        A nested dict mapping each sensor ID to its air quality trend analysis.
    """

    if isinstance(sensor_data, str):
        try:
            sensor_data = json.loads(sensor_data)
        except Exception as e:
            logging.error(f"Error parsing JSON: {e}")
            return {"error": "Invalid JSON"}

    response = {}

    for sensor_id, sensor_types in sensor_data.items():
        if target_sensor not in sensor_types:
            response[sensor_id] = {
                target_sensor: {"message": f"No data found for {target_sensor}."}
            }
            continue

        sensor_info = sensor_types.get(target_sensor, {})
        timeseries_data = sensor_info.get("timeseries_data", [])

        try:
            df = pd.DataFrame(timeseries_data)
            if "datetime" in df.columns:
                df.rename(columns={"datetime": "timestamp"}, inplace=True)
            df["timestamp"] = pd.to_datetime(df["timestamp"])
        except Exception as e:
            logging.error(f"Data error for sensor {sensor_id}: {e}")
            response[sensor_id] = {target_sensor: {"error": "Data format issue."}}
            continue

        if df.empty:
            response[sensor_id] = {target_sensor: {"message": "No data available."}}
            continue

        norm = float(df["reading_value"].mean())
        latest_value = float(df.sort_values(by="timestamp").iloc[-1]["reading_value"])

        if latest_value > norm:
            trend = "rising"
        elif latest_value < norm:
            trend = "falling"
        else:
            trend = "stable"

        response[sensor_id] = {
            target_sensor: {
                "norm": round(norm, 2),
                "latest_reading": round(latest_value, 2),
                "trend": trend,
                "message": f"{target_sensor} trend is {trend} compared to average.",
            }
        }

    if not response:
        return {"message": f"No trend analysis found for {target_sensor}."}

    return response

In [33]:
sample_json = json_data
result = analyze_air_quality_trends(sample_json)
print(json.dumps(result, indent=2))

{
  "2": {
    "Air_Quality_Sensor": {
      "norm": 10.04,
      "latest_reading": 10.0,
      "trend": "falling",
      "message": "Air_Quality_Sensor trend is falling compared to average."
    }
  }
}


In [ ]:
def analyze_hvac_anomalies(sensor_data):
    """
    Analyzes HVAC sensor data to detect anomalies in the past week.

    Expected input: either a Python dictionary or a JSON string in the following nested format:

    {
      "1": {
          "HVAC_1": {
             "timeseries_data": [
                {"datetime": "2025-02-10 05:31:59", "reading_value": 27.99, "sensor_id": "HVAC_1"},
                {"datetime": "2025-02-10 05:32:11", "reading_value": 27.99, "sensor_id": "HVAC_1"},
                ...
             ]
          },
          "Other_Sensor": { ... }
      },
      "2": {
          "HVAC_2": {
             "timeseries_data": [
                {"datetime": "2025-02-10 05:35:00", "reading_value": 28.05, "sensor_id": "HVAC_2"},
                {"datetime": "2025-02-10 05:35:12", "reading_value": 28.07, "sensor_id": "HVAC_2"},
                ...
             ]
          }
      }
    }

    For each sensor type (only those whose sensor type name contains "HVAC", case-insensitive):
      - Converts the list of readings from "timeseries_data" into a DataFrame.
      - Renames "datetime" to "timestamp" (if present) and converts it to datetime objects.
      - Filters the readings to only include data from the past 7 days.
      - Calculates the 25th percentile (Q1), 75th percentile (Q3), and IQR.
      - Identifies outliers where reading_value is below (Q1 - 1.5*IQR) or above (Q3 + 1.5*IQR).

    Returns:
      A dictionary with each HVAC sensor's identifier (taken from the sensor type key) as a key and a summary of the anomaly analysis as its value.
      For example:

      {
          "HVAC_1": {
              "anomaly_count": 3,
              "message": "Sensor HVAC_1 detected 3 anomalies in the past week."
          },
          "HVAC_2": {
              "message": "No significant anomalies detected in the HVAC system."
          }
      }
    """
    # Parse JSON if necessary.
    if isinstance(sensor_data, str):
        try:
            sensor_data = json.loads(sensor_data)
        except Exception as e:
            logging.error(f"Error parsing sensor_data JSON: {e}")
            return {"error": "Invalid sensor_data JSON"}

    # If sensor_data is provided as a list (data for one sensor), wrap it in a dict.
    if isinstance(sensor_data, list):
        sensor_data = {"1": sensor_data}

    response = {}
    now = pd.Timestamp.now()

    # Iterate over each outer sensor key.
    for outer_key, sensor_types in sensor_data.items():
        # Iterate over each sensor type within the outer key.
        for sensor_type, sensor_info in sensor_types.items():
            # Process only sensor types whose name contains "HVAC" (case-insensitive).
            if "HVAC" not in sensor_type.upper():
                continue

            timeseries_data = sensor_info.get("timeseries_data", [])
            try:
                df = pd.DataFrame(timeseries_data)
                if "datetime" in df.columns:
                    df = df.rename(columns={"datetime": "timestamp"})
                df["timestamp"] = pd.to_datetime(df["timestamp"])
            except Exception as e:
                logging.error(f"Data conversion error for sensor {sensor_type}: {e}")
                response[sensor_type] = {"error": "Invalid sensor data format"}
                continue

            # Filter data for the past 7 days.
            df = df[df["timestamp"] >= now - pd.Timedelta(days=7)]
            if df.empty:
                response[sensor_type] = {
                    "message": "No HVAC data available for the past week."
                }
                continue

            # Compute quartiles and IQR.
            Q1 = df["reading_value"].quantile(0.25)
            Q3 = df["reading_value"].quantile(0.75)
            IQR = Q3 - Q1

            # Identify outliers.
            outliers = df[
                (df["reading_value"] < Q1 - 1.5 * IQR)
                | (df["reading_value"] > Q3 + 1.5 * IQR)
            ]

            if not outliers.empty:
                response[sensor_type] = {
                    "anomaly_count": int(len(outliers)),
                    "message": f"Sensor {sensor_type} detected {len(outliers)} anomalies in the past week.",
                }
            else:
                response[sensor_type] = {
                    "message": "No significant anomalies detected in the HVAC system."
                }

    if not response:
        return {"message": "No HVAC sensor data available."}

    return response

In [66]:
import json
import logging
import pandas as pd

# Assume analyze_hvac_anomalies function is already defined as shown previously.

# Example nested sensor data for HVAC anomaly analysis
example_sensor_data = {
    "1": {
        "HVAC_1": {
            "timeseries_data": [
                {"datetime": "2025-02-10 05:31:59", "reading_value": 27.99},
                {"datetime": "2025-02-10 05:32:11", "reading_value": 29.99},
                {"datetime": "2025-02-10 05:33:00", "reading_value": 25.50},
                {"datetime": "2025-02-10 05:34:00", "reading_value": 30.00},
                {"datetime": "2025-02-10 05:35:00", "reading_value": 31.50},
            ]
        },
        "Temperature_Sensor": {
            "timeseries_data": [
                {"datetime": "2025-02-10 05:31:59", "reading_value": 22.0},
                {"datetime": "2025-02-10 05:32:11", "reading_value": 23.5},
            ]
        },
    },
    "2": {
        "HVAC_1": {
            "timeseries_data": [
                {"datetime": "2025-02-10 05:36:00", "reading_value": 28.50},
                {"datetime": "2025-02-10 05:37:00", "reading_value": 27.00},
                {"datetime": "2025-02-10 05:38:00", "reading_value": 26.50},
                {"datetime": "2025-02-10 05:39:00", "reading_value": 25.75},
                {"datetime": "2025-02-10 05:40:00", "reading_value": 24.50},
            ]
        }
    },
}

# Call the analyze_hvac_anomalies function using the example data.
hvac_anomalies = analyze_hvac_anomalies(example_sensor_data)

# Print the results.
print("HVAC Anomalies:")
for sensor, anomaly_list in hvac_anomalies.items():
    print(f"{sensor}: {anomaly_list}")

HVAC Anomalies:
HVAC_1: {'message': 'No HVAC data available for the past week.'}


In [65]:
result = analyze_hvac_anomalies(json_data)
print(json.dumps(result, indent=2))

{
  "message": "No HVAC sensor data available."
}


In [ ]:
def analyze_supply_return_temp_difference(sensor_data):
    """
    Compares supply and return air temperature sensor data from a single nested JSON structure
    and calculates the average difference.

    Expected input (as a dict or JSON string):
      {
          "1": {
              "Supply_Air_Temperature_Sensor": {
                  "timeseries_data": [
                      {"datetime": "2025-02-10 05:31:59", "reading_value": 28.5},
                      {"datetime": "2025-02-10 05:32:11", "reading_value": 29.0},
                      {"datetime": "2025-02-10 05:33:00", "reading_value": 28.0}
                  ]
              },
              "Return_Air_Temperature_Sensor": {
                  "timeseries_data": [
                      {"datetime": "2025-02-10 05:31:59", "reading_value": 27.0},
                      {"datetime": "2025-02-10 05:32:11", "reading_value": 26.5},
                      {"datetime": "2025-02-10 05:33:00", "reading_value": 27.0}
                  ]
              }
          },
          "2": {
              "Supply_Air_Temperature_Sensor": {
                  "timeseries_data": [
                      {"datetime": "2025-02-10 05:34:00", "reading_value": 23.5},
                      {"datetime": "2025-02-10 05:35:00", "reading_value": 22.5},
                      {"datetime": "2025-02-10 05:36:00", "reading_value": 21.5}
                  ]
              },
              "Return_Air_Temperature_Sensor": {
                  "timeseries_data": [
                      {"datetime": "2025-02-10 05:34:00", "reading_value": 27.5},
                      {"datetime": "2025-02-10 05:35:00", "reading_value": 27.0},
                      {"datetime": "2025-02-10 05:36:00", "reading_value": 27.0}
                  ]
              }
          }
      }

    The function aggregates readings for "Supply_Air_Temperature_Sensor" and "Return_Air_Temperature_Sensor"
    across all sensor IDs, calculates the average reading for each, computes their difference, and returns a summary.

    Returns:
      A dictionary with:
        - average_supply_temperature: Average supply temperature.
        - average_return_temperature: Average return temperature.
        - temperature_difference: Difference (supply minus return).
        - message: A descriptive message.
    """
    # Parse sensor_data if it is a JSON string.
    if isinstance(sensor_data, str):
        try:
            sensor_data = json.loads(sensor_data)
        except Exception as e:
            logging.error(f"Error parsing sensor_data JSON: {e}")
            return {"error": "Invalid sensor_data JSON"}

    # Initialize lists to collect readings.
    supply_readings = []
    return_readings = []

    # Iterate over each sensor ID and aggregate readings.
    for sensor_id, sensor_types in sensor_data.items():
        if "Supply_Air_Temperature_Sensor" in sensor_types:
            supply_readings.extend(
                sensor_types["Supply_Air_Temperature_Sensor"].get("timeseries_data", [])
            )
        if "Return_Air_Temperature_Sensor" in sensor_types:
            return_readings.extend(
                sensor_types["Return_Air_Temperature_Sensor"].get("timeseries_data", [])
            )

    if not supply_readings:
        return {"error": "No supply air temperature data found"}
    if not return_readings:
        return {"error": "No return air temperature data found"}

    try:
        # Process supply data.
        df_supply = pd.DataFrame(supply_readings)
        if "datetime" in df_supply.columns:
            df_supply = df_supply.rename(columns={"datetime": "timestamp"})
        df_supply["timestamp"] = pd.to_datetime(df_supply["timestamp"])
        df_supply = df_supply.sort_values(by="timestamp")
        avg_supply = df_supply["reading_value"].mean()

        # Process return data.
        df_return = pd.DataFrame(return_readings)
        if "datetime" in df_return.columns:
            df_return = df_return.rename(columns={"datetime": "timestamp"})
        df_return["timestamp"] = pd.to_datetime(df_return["timestamp"])
        df_return = df_return.sort_values(by="timestamp")
        avg_return = df_return["reading_value"].mean()

    except Exception as e:
        logging.error(f"Error processing temperature data: {e}")
        return {"error": "Data conversion error"}

    diff = avg_supply - avg_return
    result = {
        "average_supply_temperature": round(avg_supply, 2),
        "average_return_temperature": round(avg_return, 2),
        "temperature_difference": round(diff, 2),
        "message": (
            f"Average supply temperature is {avg_supply:.2f}°C, "
            f"average return temperature is {avg_return:.2f}°C, with a difference of {diff:.2f}°C."
        ),
    }
    return result

{
  "average_supply_temperature": 25.5,
  "average_return_temperature": 27.0,
  "temperature_difference": -1.5,
  "message": "Average supply temperature is 25.50\u00b0C, average return temperature is 27.00\u00b0C, with a difference of -1.50\u00b0C."
}


In [89]:
# Example usage:
if __name__ == "__main__":
    # Combined nested JSON structure with both supply and return temperature sensor data.
    combined_sensor_data = {
        "1": {
            "Supply_Air_Temperature_Sensor": {
                "timeseries_data": [
                    {"datetime": "2025-02-10 05:31:59", "reading_value": 28.5},
                    {"datetime": "2025-02-10 05:32:11", "reading_value": 29.0},
                    {"datetime": "2025-02-10 05:33:00", "reading_value": 28.0},
                ]
            },
            "Return_Air_Temperature_Sensor": {
                "timeseries_data": [
                    {"datetime": "2025-02-10 05:31:59", "reading_value": 27.0},
                    {"datetime": "2025-02-10 05:32:11", "reading_value": 26.5},
                    {"datetime": "2025-02-10 05:33:00", "reading_value": 27.0},
                ]
            },
        }
        # "2": {
        #     "Supply_Air_Temperature_Sensor": {
        #         "timeseries_data": [
        #             {"datetime": "2025-02-10 05:34:00", "reading_value": 23.5},
        #             {"datetime": "2025-02-10 05:35:00", "reading_value": 22.5},
        #             {"datetime": "2025-02-10 05:36:00", "reading_value": 21.5},
        #         ]
        #     },
        #     "Return_Air_Temperature_Sensor": {
        #         "timeseries_data": [
        #             {"datetime": "2025-02-10 05:34:00", "reading_value": 27.5},
        #             {"datetime": "2025-02-10 05:35:00", "reading_value": 27.0},
        #             {"datetime": "2025-02-10 05:36:00", "reading_value": 27.0},
        #         ]
        #     },
        # },
    }

    # Call the function with the combined data.
    result = analyze_supply_return_temp_difference(combined_sensor_data)

    # Print the result.
    print(json.dumps(result, indent=2))

{
  "average_supply_temperature": 28.5,
  "average_return_temperature": 26.83,
  "temperature_difference": 1.67,
  "message": "Average supply temperature is 28.50\u00b0C, average return temperature is 26.83\u00b0C, with a difference of 1.67\u00b0C."
}


In [40]:
import json
import logging
import pandas as pd


def analyze_air_flow_variation(sensor_data, target_sensor="Air_Flow_Sensor"):
    """
    Analyzes airflow variation for the specified target sensor from a nested JSON structure.

    Parameters:
      - sensor_data: JSON string or dict with structure:
        {
            "1": {
                "Air_Flow_Sensor": {
                    "timeseries_data": [
                        {"datetime": "2025-02-10 05:31:59", "reading_value": 27.99},
                        ...
                    ]
                }
            },
            "2": { ... }
        }

      - target_sensor: default "Air_Flow_Sensor", can be customized if needed.

    For each timeseries ID:
      - Extracts data for target_sensor.
      - Computes mean, std deviation, coefficient of variation.
      - Reports stability/instability based on CV.

    Returns:
      A nested dict mapping each timeseries ID to results for the target_sensor.
    """
    if isinstance(sensor_data, str):
        try:
            sensor_data = json.loads(sensor_data)
        except Exception as e:
            logging.error(f"Error parsing sensor_data JSON: {e}")
            return {"error": "Invalid JSON input"}

    response = {}

    for sensor_id, sensor_types in sensor_data.items():
        if target_sensor not in sensor_types:
            response[sensor_id] = {
                target_sensor: {"message": f"No data available for {target_sensor}."}
            }
            continue

        timeseries_data = sensor_types[target_sensor].get("timeseries_data", [])
        if not timeseries_data:
            response[sensor_id] = {
                target_sensor: {"message": f"No readings found for {target_sensor}."}
            }
            continue

        try:
            df = pd.DataFrame(timeseries_data)
            if "datetime" in df.columns:
                df.rename(columns={"datetime": "timestamp"}, inplace=True)
            df["timestamp"] = pd.to_datetime(df["timestamp"])
        except Exception as e:
            logging.error(f"Data conversion error for sensor {sensor_id}: {e}")
            response[sensor_id] = {target_sensor: {"error": "Data formatting error"}}
            continue

        mean_val = df["reading_value"].mean()
        std_val = df["reading_value"].std() or 0.0
        cv = std_val / mean_val if mean_val else 0

        response[sensor_id] = {
            target_sensor: {
                "mean_airflow": round(mean_val, 2),
                "std_dev_airflow": round(std_val, 2),
                "coefficient_of_variation": round(cv, 2),
                "message": (
                    f"{target_sensor} coefficient of variation: {cv:.2f}. "
                    + ("Stable airflow." if cv < 0.1 else "High variation detected.")
                ),
            }
        }

    return response

In [44]:
if __name__ == "__main__":
    sample_json = {
        "1": {
            "Air_Flow_Sensor": {
                "timeseries_data": [
                    {"datetime": "2025-02-10 05:31:59", "reading_value": 100},
                    {"datetime": "2025-02-10 05:32:59", "reading_value": 98},
                    {"datetime": "2025-02-10 05:33:59", "reading_value": 101},
                ]
            }
        },
        "2": {
            "Air_Flow_Sensor": {
                "timeseries_data": [
                    {"datetime": "2025-02-10 05:31:59", "reading_value": 200},
                    {"datetime": "2025-02-10 05:32:59", "reading_value": 210},
                    {
                        "datetime": "2025-02-10 05:33:59",
                        "reading_value": 250,
                    },  # Large variation
                ]
            }
        },
    }

    result = analyze_air_flow_variation(sample_json)  # Defaults to Air_Flow_Sensor
    print(json.dumps(result, indent=2))

{
  "1": {
    "Air_Flow_Sensor": {
      "mean_airflow": 99.67,
      "std_dev_airflow": 1.53,
      "coefficient_of_variation": 0.02,
      "message": "Air_Flow_Sensor coefficient of variation: 0.02. Stable airflow."
    }
  },
  "2": {
    "Air_Flow_Sensor": {
      "mean_airflow": 220.0,
      "std_dev_airflow": 26.46,
      "coefficient_of_variation": 0.12,
      "message": "Air_Flow_Sensor coefficient of variation: 0.12. High variation detected."
    }
  }
}


In [ ]:
def analyze_pressure_trend(sensor_data, expected_range=(0.5, 1.5)):
    """
    Analyzes static pressure sensor data to check if average readings are within an expected range,
    accepting a nested JSON structure.

    Expected input (as a Python dict or JSON string):
      {
          "1": {
              "Static_Pressure_Sensor": {
                  "timeseries_data": [
                      {"datetime": "2025-02-10 05:31:59", "reading_value": 1.2},
                      {"datetime": "2025-02-10 05:32:11", "reading_value": 1.3},
                      ...
                  ]
              }
          },
          "2": {
              "Static_Pressure_Sensor": {
                  "timeseries_data": [
                      {"datetime": "2025-02-10 05:35:00", "reading_value": 0.8},
                      {"datetime": "2025-02-10 05:35:12", "reading_value": 0.7},
                      ...
                  ]
              }
          }
      }

    For each sensor type:
      - Converts the list of readings (from "timeseries_data") into a DataFrame.
      - Renames "datetime" to "timestamp" (if necessary) and converts it to datetime objects.
      - Computes the average static pressure.
      - Compares it with the expected_range.

    Returns:
      A nested dictionary where each sensor ID maps to sensor type keys with their analysis.
    """
    # Parse JSON string if necessary.
    if isinstance(sensor_data, str):
        try:
            sensor_data = json.loads(sensor_data)
        except Exception as e:
            logging.error(f"Error parsing sensor_data JSON: {e}")
            return {"error": "Invalid sensor_data JSON"}

    # If sensor_data is a list (data for one sensor), wrap it in a dictionary.
    if isinstance(sensor_data, list):
        sensor_data = {"1": sensor_data}

    response = {}
    for sensor_id, sensor_types in sensor_data.items():
        response[sensor_id] = {}
        for sensor_type, sensor_info in sensor_types.items():
            # Get the readings from the "timeseries_data" key.
            timeseries_data = sensor_info.get("timeseries_data", [])
            try:
                df = pd.DataFrame(timeseries_data)
                # Rename "datetime" to "timestamp" if present.
                if "datetime" in df.columns:
                    df = df.rename(columns={"datetime": "timestamp"})
                df["timestamp"] = pd.to_datetime(df["timestamp"])
            except Exception as e:
                logging.error(
                    f"Data processing error for sensor {sensor_id}, type {sensor_type}: {e}"
                )
                response[sensor_id][sensor_type] = {
                    "error": "Invalid sensor data format"
                }
                continue

            if df.empty:
                response[sensor_id][sensor_type] = {
                    "message": "No data available for this sensor."
                }
                continue

            avg_pressure = df["reading_value"].mean()

            if expected_range[0] <= avg_pressure <= expected_range[1]:
                message = (
                    f"Sensor {sensor_id} ({sensor_type}) average pressure "
                    f"{avg_pressure:.2f} is within the expected range."
                )
                status = "normal"
            else:
                message = (
                    f"Sensor {sensor_id} ({sensor_type}) average pressure "
                    f"{avg_pressure:.2f} is out of the expected range {expected_range}."
                )
                status = "abnormal"

            response[sensor_id][sensor_type] = {
                "average_pressure": round(avg_pressure, 2),
                "status": status,
                "message": message,
            }

    if not response:
        return {"message": "No pressure sensor data found."}

    return response

In [96]:
# Example dummy input for two static pressure sensors:
# dummy_pressure_data = {
#     "static_pressure_sensor_1": [
#         {"datetime": "2025-03-18 10:00:00", "reading_value": 1.2},
#         {"datetime": "2025-03-18 10:05:00", "reading_value": 1.3},
#         {"datetime": "2025-03-18 10:10:00", "reading_value": 1.1},
#         {"datetime": "2025-03-18 10:15:00", "reading_value": 1.25},
#     ],
#     "static_pressure_sensor_2": [
#         {"datetime": "2025-03-18 10:00:00", "reading_value": 0.3},
#         {"datetime": "2025-03-18 10:05:00", "reading_value": 0.4},
#         {"datetime": "2025-03-18 10:10:00", "reading_value": 0.35},
#         {"datetime": "2025-03-18 10:15:00", "reading_value": 0.38},
#     ],
# }

# Run the pressure trend analysis
result = analyze_pressure_trend(json_data, expected_range=(0.5, 1.5))

# Display the result in a readable JSON format
print(json.dumps(result, indent=2))

{
  "2": {
    "Air_Temperature_Sensor": {
      "average_pressure": 28.99,
      "status": "abnormal",
      "message": "Sensor 2 (Air_Temperature_Sensor) average pressure 28.99 is out of the expected range (0.5, 1.5)."
    },
    "Zone_Air_Humidity_Sensor": {
      "average_pressure": 23.22,
      "status": "abnormal",
      "message": "Sensor 2 (Zone_Air_Humidity_Sensor) average pressure 23.22 is out of the expected range (0.5, 1.5)."
    }
  }
}


## Environmental monitonring system


In [97]:
def analyze_sensor_trend(sensor_data, window=3):
    """
    Analyzes the trend of sensor readings using a moving average from a nested JSON structure.

    Expected input (as a Python dict or JSON string):
      {
          "1": {
              "Sensor_Type_A": {
                  "timeseries_data": [
                      {"datetime": "2025-02-10 05:31:59", "reading_value": 27.99},
                      {"datetime": "2025-02-10 05:32:11", "reading_value": 27.99},
                      ...
                  ]
              },
              "Sensor_Type_B": { ... }
          },
          "2": {
              "Sensor_Type_A": { ... }
          }
      }

    For each sensor type:
      - Converts the list of readings (from "timeseries_data") into a DataFrame.
      - Renames "datetime" to "timestamp" (if necessary) and converts it to datetime objects.
      - Sorts by timestamp and computes a rolling average with the specified window.
      - Determines the trend as "upward", "downward", or "stable" by comparing the first and last rolling mean.

    Returns:
      A nested dictionary where each sensor ID maps to sensor type keys with their trend analysis details.
    """
    # Parse JSON string if necessary.
    if isinstance(sensor_data, str):
        try:
            sensor_data = json.loads(sensor_data)
        except Exception as e:
            logging.error(f"Error parsing sensor_data JSON: {e}")
            return {"error": "Invalid sensor_data JSON"}

    # If sensor_data is provided as a list, wrap it in a dict with a default key "1".
    if isinstance(sensor_data, list):
        sensor_data = {"1": sensor_data}

    response = {}
    trend_threshold = 0.05  # threshold to decide if change is significant (adjustable)

    # Process each sensor ID.
    for sensor_id, sensor_types in sensor_data.items():
        response[sensor_id] = {}
        # Process each sensor type for the current sensor ID.
        for sensor_type, sensor_info in sensor_types.items():
            timeseries_data = sensor_info.get("timeseries_data", [])
            try:
                # Convert list of readings into a DataFrame.
                df = pd.DataFrame(timeseries_data)
                # Rename "datetime" to "timestamp" if needed.
                if "datetime" in df.columns:
                    df = df.rename(columns={"datetime": "timestamp"})
                df["timestamp"] = pd.to_datetime(df["timestamp"])
                df = df.sort_values(by="timestamp")
                # Compute rolling average.
                df["rolling_mean"] = (
                    df["reading_value"].rolling(window=window, min_periods=1).mean()
                )
            except Exception as e:
                logging.error(
                    f"Data conversion error for sensor {sensor_id}, type {sensor_type}: {e}"
                )
                response[sensor_id][sensor_type] = {
                    "error": "Invalid sensor data format"
                }
                continue

            # Compute trend using difference between the first and last rolling average values.
            trend_diff = df["rolling_mean"].iloc[-1] - df["rolling_mean"].iloc[0]
            if abs(trend_diff) < trend_threshold:
                trend = "stable"
            elif trend_diff > 0:
                trend = "upward"
            else:
                trend = "downward"

            response[sensor_id][sensor_type] = {
                "initial_rolling_mean": df["rolling_mean"].iloc[0],
                "latest_rolling_mean": df["rolling_mean"].iloc[-1],
                "trend": trend,
                "difference": trend_diff,
            }

    return response

In [98]:
# Example usage:
if __name__ == "__main__":
    # Example JSON input as a string
    # sample_json = """
    # {
    #     "1": [
    #         {"datetime": "2025-02-10 05:31:59", "reading_value": 27.99},
    #         {"datetime": "2025-02-10 05:32:11", "reading_value": 27.99},
    #         {"datetime": "2025-02-10 05:32:22", "reading_value": 27.98}
    #     ],
    #     "2": [
    #         {"datetime": "2025-02-10 05:35:00", "reading_value": 28.05},
    #         {"datetime": "2025-02-10 05:35:12", "reading_value": 28.07},
    #         {"datetime": "2025-02-10 05:35:25", "reading_value": 28.10}
    #     ]
    # }
    # """
    # or use previous output
    sample_json = json_data
    result = analyze_sensor_trend(sample_json)
    print(json.dumps(result, indent=2))

{
  "2": {
    "Air_Temperature_Sensor": {
      "initial_rolling_mean": 29.06,
      "latest_rolling_mean": 28.91,
      "trend": "downward",
      "difference": -0.14999999999999858
    },
    "Zone_Air_Humidity_Sensor": {
      "initial_rolling_mean": 23.37,
      "latest_rolling_mean": 23.113333333333333,
      "trend": "downward",
      "difference": -0.2566666666666677
    }
  }
}


In [ ]:
def aggregate_sensor_data(sensor_data, freq="H"):
    """
    Aggregates sensor data into defined time intervals (e.g., hourly, daily) and computes summary statistics,
    accepting a nested JSON structure.

    Expected input (as a Python dict or JSON string):
      {
          "1": {
              "Sensor_Type_A": {
                  "timeseries_data": [
                      {"datetime": "2025-02-10 05:31:59", "reading_value": 27.99},
                      {"datetime": "2025-02-10 05:32:11", "reading_value": 27.99},
                      ...
                  ]
              },
              "Sensor_Type_B": { ... }
          },
          "2": {
              "Sensor_Type_A": { ... }
          }
      }

    Returns:
      A nested dictionary mapping sensor IDs to sensor type keys and their aggregated summaries.
      Each summary (list of records) includes the mean, standard deviation, minimum, and maximum values,
      with timestamps converted to string format.
    """
    # Parse JSON string if needed.
    if isinstance(sensor_data, str):
        try:
            sensor_data = json.loads(sensor_data)
        except Exception as e:
            logging.error(f"Error parsing sensor_data JSON: {e}")
            return {"error": "Invalid sensor_data JSON"}

    aggregated_results = {}

    # Iterate over each sensor ID.
    for sensor_id, sensor_types in sensor_data.items():
        aggregated_results[sensor_id] = {}
        # Iterate over each sensor type within this sensor ID.
        for sensor_type, sensor_info in sensor_types.items():
            timeseries_data = sensor_info.get("timeseries_data", [])
            try:
                df = pd.DataFrame(timeseries_data)
                # Rename "datetime" column to "timestamp" if necessary.
                if "datetime" in df.columns:
                    df = df.rename(columns={"datetime": "timestamp"})
                df["timestamp"] = pd.to_datetime(df["timestamp"])
                df = df.sort_values(by="timestamp")
                df.set_index("timestamp", inplace=True)
                # Resample and compute summary statistics.
                agg_df = (
                    df["reading_value"]
                    .resample(freq)
                    .agg(["mean", "std", "min", "max"])
                )
                # Reset index and convert timestamp to string.
                agg_df = agg_df.reset_index()
                agg_df["timestamp"] = agg_df["timestamp"].dt.strftime(
                    "%Y-%m-%d %H:%M:%S"
                )
                aggregated_results[sensor_id][sensor_type] = agg_df.to_dict(
                    orient="records"
                )
            except Exception as e:
                logging.error(
                    f"Aggregation error for sensor {sensor_id}, type {sensor_type}: {e}"
                )
                aggregated_results[sensor_id][sensor_type] = {
                    "error": "Aggregation failed"
                }

    return aggregated_results

In [100]:
# Example usage:
if __name__ == "__main__":
    # Example JSON input as a string
    # sample_sensor_data = """
    # {
    #     "1": [
    #         {"datetime": "2025-02-10 05:31:59", "reading_value": 27.99},
    #         {"datetime": "2025-02-10 05:32:11", "reading_value": 27.99},
    #         {"datetime": "2025-02-10 05:32:22", "reading_value": 27.98}
    #     ],
    #     "2": [
    #         {"datetime": "2025-02-10 05:35:00", "reading_value": 28.05},
    #         {"datetime": "2025-02-10 05:35:12", "reading_value": 28.07},
    #         {"datetime": "2025-02-10 05:35:25", "reading_value": 28.10}
    #     ]
    # }
    # """

    # or use previous output
    # sample_sensor_data = output
    aggregated_results = aggregate_sensor_data(
        json_data, freq="min"
    )  # using minute-level for demo
    print("\nAggregated Data:")
    print(json.dumps(aggregated_results, indent=2))


Aggregated Data:
{
  "2": {
    "Air_Temperature_Sensor": [
      {
        "timestamp": "2025-02-10 00:00:00",
        "mean": 29.05666666666667,
        "std": 0.010327955589886282,
        "min": 29.04,
        "max": 29.07
      },
      {
        "timestamp": "2025-02-10 00:01:00",
        "mean": 29.05,
        "std": 0.0070710678118649575,
        "min": 29.04,
        "max": 29.06
      },
      {
        "timestamp": "2025-02-10 00:02:00",
        "mean": 29.05,
        "std": 0.015811388300841604,
        "min": 29.03,
        "max": 29.07
      },
      {
        "timestamp": "2025-02-10 00:03:00",
        "mean": 29.044,
        "std": 0.008944271909999787,
        "min": 29.03,
        "max": 29.05
      },
      {
        "timestamp": "2025-02-10 00:04:00",
        "mean": 29.036666666666665,
        "std": 0.012110601416389767,
        "min": 29.02,
        "max": 29.05
      },
      {
        "timestamp": "2025-02-10 00:05:00",
        "mean": 29.044,
        "std": 0

In [ ]:
def correlate_sensors(sensor_data_dict):
    """
    Computes the correlation matrix among multiple sensors from a nested JSON structure.

    Expected input (as a Python dict or JSON string):
      {
          "1": {
              "Air_Temperature_Sensor": {
                  "timeseries_data": [
                      {"datetime": "2025-02-10 05:31:59", "reading_value": 27.99},
                      {"datetime": "2025-02-10 05:32:11", "reading_value": 27.99},
                      ...
                  ]
              },
              "Zone_Air_Humidity_Sensor": {
                  "timeseries_data": [
                      {"datetime": "2025-02-10 05:35:00", "reading_value": 28.05},
                      {"datetime": "2025-02-10 05:35:12", "reading_value": 28.07},
                      ...
                  ]
              }
          },
          "2": {
              "Air_Temperature_Sensor": {
                  "timeseries_data": [
                      {"datetime": "2025-02-10 06:00:00", "reading_value": 28.10},
                      ...
                  ]
              }
          }
      }

    The function flattens the nested structure (using keys like "1_Air_Temperature_Sensor") and
    merges sensor data on timestamps (with a tolerance of 1 minute), then computes the Pearson
    correlation between sensor readings.

    Returns:
      A correlation matrix as a nested dictionary.
    """
    # Parse JSON string to dict if needed.
    if isinstance(sensor_data_dict, str):
        try:
            sensor_data_dict = json.loads(sensor_data_dict)
        except Exception as e:
            logging.error(f"Error parsing sensor_data JSON: {e}")
            return {"error": "Invalid sensor_data JSON"}

    dfs = []

    # Flatten the nested structure:
    for sensor_id, sensor_types in sensor_data_dict.items():
        for sensor_type, sensor_info in sensor_types.items():
            timeseries_data = sensor_info.get("timeseries_data", [])
            try:
                df = pd.DataFrame(timeseries_data)
                if "datetime" in df.columns:
                    df = df.rename(columns={"datetime": "timestamp"})
                df["timestamp"] = pd.to_datetime(df["timestamp"])
                df = df.sort_values(by="timestamp")
                # Rename the reading_value column to a unique sensor name
                unique_name = f"{sensor_id}_{sensor_type}"
                df = df[["timestamp", "reading_value"]].rename(
                    columns={"reading_value": unique_name}
                )
                dfs.append(df)
            except Exception as e:
                logging.error(
                    f"Error processing sensor {sensor_id} ({sensor_type}): {e}"
                )

    if not dfs:
        return {"error": "No valid sensor data to correlate."}

    # Merge all DataFrames on the timestamp column using asof merge with a tolerance of 1 minute.
    merged_df = dfs[0]
    for df in dfs[1:]:
        merged_df = pd.merge_asof(
            merged_df,
            df,
            on="timestamp",
            tolerance=pd.Timedelta("1min"),
            direction="nearest",
        )

    # Drop the timestamp column and compute the correlation matrix.
    corr_matrix = merged_df.drop(columns=["timestamp"]).corr(method="pearson")
    return corr_matrix.to_dict()

In [102]:
# Example usage:
if __name__ == "__main__":
    # Example JSON input as a string
    # sample_json = """
    # {
    #     "1": [
    #         {"datetime": "2025-02-10 05:31:59", "reading_value": 27.99},
    #         {"datetime": "2025-02-10 05:32:11", "reading_value": 27.99},
    #         {"datetime": "2025-02-10 05:32:22", "reading_value": 27.98}
    #     ],
    #     "2": [
    #         {"datetime": "2025-02-10 05:35:00", "reading_value": 28.05},
    #         {"datetime": "2025-02-10 05:35:12", "reading_value": 28.07},
    #         {"datetime": "2025-02-10 05:35:25", "reading_value": 28.10}
    #     ]
    # }
    # """
    # or use previous output
    sample_json = json_data
    result = correlate_sensors(sample_json)
    print(json.dumps(result, indent=2))

{
  "2_Air_Temperature_Sensor": {
    "2_Air_Temperature_Sensor": 1.0,
    "2_Zone_Air_Humidity_Sensor": 0.7806850635254113
  },
  "2_Zone_Air_Humidity_Sensor": {
    "2_Air_Temperature_Sensor": 0.7806850635254113,
    "2_Zone_Air_Humidity_Sensor": 1.0
  }
}


In [ ]:
def compute_air_quality_index(sensor_data):
    """
    Computes a composite Air Quality Index (AQI) based on selected pollutant sensors from a nested JSON structure.

    Expected sensor keys (if available) in the nested input:
      - "PM2.5_Level_Sensor_Standard"
      - "PM10_Level_Sensor_Standard"
      - "NO2_Level_Sensor"
      - "CO_Level_Sensor"
      - "CO2_Level_Sensor"

    Input format (as a Python dict or JSON string):
      {
          "1": {
              "PM2.5_Level_Sensor_Standard": {
                  "timeseries_data": [
                      {"datetime": "2025-02-10 05:31:59", "reading_value": 27.99},
                      ...
                  ]
              },
              "Other_Sensor": { ... }
          },
          "2": {
              "PM10_Level_Sensor_Standard": {
                  "timeseries_data": [
                      {"datetime": "2025-02-10 05:35:00", "reading_value": 45.0},
                      ...
                  ]
              },
              "NO2_Level_Sensor": {
                  "timeseries_data": [
                      {"datetime": "2025-02-10 05:35:12", "reading_value": 38.0},
                      ...
                  ]
              }
          },
          ...
      }

    For each expected sensor, the function:
      - Aggregates all readings across the nested structure.
      - Converts the data to a DataFrame, renames the "datetime" column (if present) to "timestamp" and converts it.
      - Sorts by timestamp and takes the latest reading.
      - Normalizes the reading by dividing by an arbitrary threshold.
      - Multiplies by a weight to obtain a component value.

    Finally, it sums the weighted components to compute the composite AQI and assigns a health status.
    """
    # Parse JSON string if needed.
    if isinstance(sensor_data, str):
        try:
            sensor_data = json.loads(sensor_data)
        except Exception as e:
            logging.error(f"Error parsing sensor_data JSON: {e}")
            return {"error": "Invalid sensor_data JSON"}

    # Define arbitrary thresholds and weights.
    thresholds = {
        "PM2.5_Level_Sensor_Standard": 35,
        "PM10_Level_Sensor_Standard": 50,
        "NO2_Level_Sensor": 40,
        "CO_Level_Sensor": 9,
        "CO2_Level_Sensor": 1000,
    }

    weights = {
        "PM2.5_Level_Sensor_Standard": 0.3,
        "PM10_Level_Sensor_Standard": 0.2,
        "NO2_Level_Sensor": 0.2,
        "CO_Level_Sensor": 0.15,
        "CO2_Level_Sensor": 0.15,
    }

    index_components = {}

    # For each expected sensor type, gather all readings from the nested structure.
    for sensor_type, threshold in thresholds.items():
        aggregated_readings = []
        # Loop over each outer key (sensor ID) in the nested structure.
        for sensor_id, sensor_types in sensor_data.items():
            if sensor_type in sensor_types:
                timeseries = sensor_types[sensor_type].get("timeseries_data", [])
                aggregated_readings.extend(timeseries)
        if not aggregated_readings:
            continue
        try:
            df = pd.DataFrame(aggregated_readings)
            if "datetime" in df.columns:
                df = df.rename(columns={"datetime": "timestamp"})
            df["timestamp"] = pd.to_datetime(df["timestamp"])
            df = df.sort_values(by="timestamp")
            latest_value = df.iloc[-1]["reading_value"]
            # Normalize the latest reading (simple ratio normalization).
            normalized = latest_value / threshold
            index_components[sensor_type] = normalized * weights[sensor_type]
        except Exception as e:
            logging.error(
                f"Error computing AQI component for sensor {sensor_type}: {e}"
            )

    if not index_components:
        return {"error": "Insufficient data for AQI calculation."}

    aqi = sum(index_components.values())

    # Define a simple category (arbitrary ranges for demonstration purposes)
    if aqi < 0.5:
        status = "Good"
    elif aqi < 1:
        status = "Moderate"
    elif aqi < 1.5:
        status = "Unhealthy for Sensitive Groups"
    else:
        status = "Unhealthy"

    return {"AQI": aqi, "Status": status, "Components": index_components}

In [112]:
# sample_sensor_data_aqi = {
#     "PM2.5_Level_Sensor_Standard": [
#         {"datetime": "2025-02-10 05:31:59", "reading_value": 20},
#         {"datetime": "2025-02-10 05:32:59", "reading_value": 22},
#     ],
#     "PM10_Level_Sensor_Standard": [
#         {"datetime": "2025-02-10 05:31:59", "reading_value": 30},
#         {"datetime": "2025-02-10 05:32:59", "reading_value": 32},
#     ],
#     "NO2_Level_Sensor": [
#         {"datetime": "2025-02-10 05:31:59", "reading_value": 35},
#         {"datetime": "2025-02-10 05:32:59", "reading_value": 36},
#     ],
#     "CO_Level_Sensor": [
#         {"datetime": "2025-02-10 05:31:59", "reading_value": 5},
#         {"datetime": "2025-02-10 05:32:59", "reading_value": 5.5},
#     ],
#     "CO2_Level_Sensor": [
#         {"datetime": "2025-02-10 05:31:59", "reading_value": 800},
#         {"datetime": "2025-02-10 05:32:59", "reading_value": 850},
#     ],
# }
aqi_result = compute_air_quality_index(json_data)
print("\nAir Quality Index:")
print(json.dumps(aqi_result, indent=2))


Air Quality Index:
{
  "AQI": 2.4499166666666667,
  "Status": "Unhealthy",
  "Components": {
    "PM10_Level_Sensor_Standard": 0.024,
    "NO2_Level_Sensor": 0.085,
    "CO_Level_Sensor": 2.2666666666666666,
    "CO2_Level_Sensor": 0.07425
  }
}


In [59]:
def generate_health_alerts(sensor_data, thresholds):
    """
    Generates alerts if the latest sensor readings exceed specified threshold ranges,
    accepting a nested JSON structure.

    Parameters:
      - sensor_data: A dict or JSON string in the following format:
          {
              "1": {
                  "PM2.5_Level_Sensor_Standard": {
                      "timeseries_data": [
                          {"datetime": "2025-02-10 05:31:59", "reading_value": 27.99},
                          {"datetime": "2025-02-10 05:32:11", "reading_value": 27.99},
                          ...
                      ]
                  },
                  "Other_Sensor": { ... }
              },
              "2": {
                  "NO2_Level_Sensor": {
                      "timeseries_data": [
                          {"datetime": "2025-02-10 05:35:00", "reading_value": 38.0},
                          {"datetime": "2025-02-10 05:35:12", "reading_value": 39.5},
                          ...
                      ]
                  }
              }
          }
      - thresholds: A dict mapping sensor names (as used in the inner keys) to a tuple (min_value, max_value).

    For each sensor type specified in thresholds, the function finds the latest reading and returns an alert
    if the reading is below min_value or above max_value. The resulting alerts are keyed by a unique identifier
    in the format "sensorID_sensorType".

    Returns a dictionary with alert messages per sensor.
    """
    # Convert sensor_data from JSON string to dict if needed.
    if isinstance(sensor_data, str):
        try:
            sensor_data = json.loads(sensor_data)
        except Exception as e:
            logging.error(f"Error parsing sensor_data JSON: {e}")
            return {"error": "Invalid sensor_data JSON"}

    alerts = {}

    # Iterate over each outer sensor ID.
    for sensor_id, sensor_types in sensor_data.items():
        # Iterate over each sensor type within this sensor ID.
        for sensor_type, sensor_info in sensor_types.items():
            # Process only if sensor_type is among those in thresholds.
            if sensor_type not in thresholds:
                continue

            min_val, max_val = thresholds[sensor_type]
            readings = sensor_info.get("timeseries_data", [])
            unique_key = f"{sensor_id}_{sensor_type}"

            if not readings:
                alerts[unique_key] = "No data available."
                continue

            try:
                df = pd.DataFrame(readings)
                if "datetime" in df.columns:
                    df = df.rename(columns={"datetime": "timestamp"})
                df["timestamp"] = pd.to_datetime(df["timestamp"])
                df = df.sort_values(by="timestamp")
                latest_value = df.iloc[-1]["reading_value"]
            except Exception as e:
                logging.error(
                    f"Error processing sensor {sensor_id} ({sensor_type}): {e}"
                )
                alerts[unique_key] = "Data error."
                continue

            if latest_value < min_val or latest_value > max_val:
                alerts[unique_key] = (
                    f"Alert: Latest reading {latest_value} out of range [{min_val}, {max_val}]."
                )
            else:
                alerts[unique_key] = (
                    f"OK: Latest reading {latest_value} within acceptable range."
                )

    return alerts

In [61]:
# Example usage:
if __name__ == "__main__":
    # Example JSON input as a string
    # sample_sensor_data = """
    # {
    #     "Sound_Noise_Sensor_MEMS": [
    #         {"datetime": "2025-02-10 05:31:59", "reading_value": 65},
    #         {"datetime": "2025-02-10 05:32:59", "reading_value": 68},
    #         {"datetime": "2025-02-10 05:33:59", "reading_value": 95}
    #     ],
    #     "PM2.5_Level_Sensor_Standard": [
    #         {"datetime": "2025-02-10 05:31:59", "reading_value": 20},
    #         {"datetime": "2025-02-10 05:32:59", "reading_value": 22},
    #         {"datetime": "2025-02-10 05:33:59", "reading_value": 50}
    #     ]
    # }
    # """

    thresholds = {
        "Air_Temperature_Sensor": (20, 25),  # allowable range for noise levels
        "PM2_5_Level_Sensor_Standard": (0, 10),
        "Zone_Air_Humidity_Sensor": (30, 70),
        "PM2_5_Level_Sensor_Standard": (0, 10),
        "PM10_Level_Sensor_Standard": (0, 20),
        "NO2_Level_Sensor": (0, 50),
        "CO_Level_Sensor": (0, 10),
        "CO2_Level_Sensor": (0, 1000),
    }
    alerts = generate_health_alerts(json_data, thresholds)
    print("\nHealth Alerts:")
    print(json.dumps(alerts, indent=2))


Health Alerts:
{
  "2_Air_Temperature_Sensor": "Alert: Latest reading 28.97 out of range [20, 25].",
  "2_Zone_Air_Humidity_Sensor": "Alert: Latest reading 23.2 out of range [30, 70].",
  "2_PM2_5_Level_Sensor_Standard": "OK: Latest reading 6 within acceptable range.",
  "2_PM10_Level_Sensor_Standard": "OK: Latest reading 6 within acceptable range.",
  "2_NO2_Level_Sensor": "OK: Latest reading 17 within acceptable range.",
  "2_CO_Level_Sensor": "Alert: Latest reading 136 out of range [0, 10].",
  "2_CO2_Level_Sensor": "OK: Latest reading 519 within acceptable range."
}


In [ ]:
def detect_anomalies(sensor_data, method="zscore", threshold=3, robust=False):
    """
    Detects anomalies in sensor data using a statistical approach from a nested JSON structure.

    Parameters:
      - sensor_data: A dict or JSON string of sensor data. Expected format:
            {
                "1": {
                    "Sensor_Type_A": {
                        "timeseries_data": [
                            {"datetime": "2025-02-10 05:31:59", "reading_value": 27.99},
                            {"datetime": "2025-02-10 05:32:11", "reading_value": 27.99},
                            ...
                        ]
                    },
                    "Sensor_Type_B": { ... }
                },
                "2": { ... }
            }
      - method: Currently supports only "zscore" (standard or robust based on the `robust` flag).
      - threshold: The z-score threshold above which a reading is flagged as an anomaly.
      - robust: If True, uses the median and median absolute deviation (MAD) for z-score calculation,
                which is more robust to outliers.

    Returns:
      A dictionary mapping flattened sensor names (e.g. "1_Sensor_Type_A") to a list of anomalous readings.
      Each anomalous reading includes the timestamp, reading_value, and computed zscore.
    """
    # Convert JSON string to dict if needed.
    if isinstance(sensor_data, str):
        try:
            sensor_data = json.loads(sensor_data)
        except Exception as e:
            logging.error(f"Error parsing sensor_data JSON: {e}")
            return {"error": "Invalid sensor_data JSON"}

    anomalies = {}

    # Iterate over each sensor ID in the nested structure.
    for sensor_id, sensor_types in sensor_data.items():
        # Iterate over each sensor type.
        for sensor_type, sensor_info in sensor_types.items():
            unique_key = f"{sensor_id}_{sensor_type}"
            timeseries_data = sensor_info.get("timeseries_data", [])
            try:
                df = pd.DataFrame(timeseries_data)
                if "datetime" in df.columns:
                    df = df.rename(columns={"datetime": "timestamp"})
                df["timestamp"] = pd.to_datetime(df["timestamp"])
                df = df.sort_values(by="timestamp")

                if robust:
                    median_val = df["reading_value"].median()
                    mad = np.median(np.abs(df["reading_value"] - median_val))
                    if mad == 0:
                        mad = 1  # Avoid division by zero
                    df["zscore"] = 0.6745 * (df["reading_value"] - median_val) / mad
                else:
                    mean_val = df["reading_value"].mean()
                    std_val = df["reading_value"].std() or 1  # Avoid division by zero
                    df["zscore"] = (df["reading_value"] - mean_val) / std_val

                # Flag anomalies where the absolute z-score exceeds the threshold.
                anomaly_df = df[np.abs(df["zscore"]) > threshold].copy()
                anomaly_df = anomaly_df.assign(
                    timestamp=anomaly_df["timestamp"].dt.strftime("%Y-%m-%d %H:%M:%S")
                )
                anomalies[unique_key] = anomaly_df[
                    ["timestamp", "reading_value", "zscore"]
                ].to_dict(orient="records")
            except Exception as e:
                logging.error(f"Error detecting anomalies for sensor {unique_key}: {e}")
                anomalies[unique_key] = {
                    "error": "Anomaly detection failed",
                    "details": str(e),
                }

    return anomalies

In [118]:
# Sample usage:
if __name__ == "__main__":
    # Example JSON input as a string
    # sample_sensor_data = """
    # {
    #     "Sound_Noise_Sensor_MEMS": [
    #         {"datetime": "2025-02-10 05:31:59", "reading_value": 65},
    #         {"datetime": "2025-02-10 05:32:59", "reading_value": 68},
    #         {"datetime": "2025-02-10 05:33:59", "reading_value": 95}
    #     ],
    #     "PM2.5_Level_Sensor_Standard": [
    #         {"datetime": "2025-02-10 05:31:59", "reading_value": 20},
    #         {"datetime": "2025-02-10 05:32:59", "reading_value": 2245},
    #         {"datetime": "2025-02-10 05:33:59", "reading_value": 50}
    #     ]
    # }
    # """

    # Detect anomalies with a threshold of 1.5 using robust statistics.
    anomalies = detect_anomalies(json_data, threshold=1.5, robust=True)
    print("\nAnomaly Detection:")
    print(json.dumps(anomalies, indent=2))


Anomaly Detection:
{
  "2_Air_Temperature_Sensor": [
    {
      "timestamp": "2025-02-10 00:00:00",
      "reading_value": 29.06,
      "zscore": 1.79866666666656
    },
    {
      "timestamp": "2025-02-10 00:00:11",
      "reading_value": 29.05,
      "zscore": 1.5738333333332801
    },
    {
      "timestamp": "2025-02-10 00:00:23",
      "reading_value": 29.07,
      "zscore": 2.0234999999999204
    },
    {
      "timestamp": "2025-02-10 00:00:34",
      "reading_value": 29.06,
      "zscore": 1.79866666666656
    },
    {
      "timestamp": "2025-02-10 00:00:45",
      "reading_value": 29.06,
      "zscore": 1.79866666666656
    },
    {
      "timestamp": "2025-02-10 00:01:20",
      "reading_value": 29.05,
      "zscore": 1.5738333333332801
    },
    {
      "timestamp": "2025-02-10 00:01:31",
      "reading_value": 29.05,
      "zscore": 1.5738333333332801
    },
    {
      "timestamp": "2025-02-10 00:01:43",
      "reading_value": 29.05,
      "zscore": 1.5738333333332801

In [122]:
def analyze_noise_levels(
    sensor_data, sensor_key="Sound_Noise_Sensor_MEMS", threshold=90
):
    """
    Analyzes noise level data from a nested JSON structure.

    Aggregates readings for the specified sensor_key from across all sensor IDs.
    Computes the mean, min, max, standard deviation and flags if the latest reading exceeds the threshold.

    Parameters:
      - sensor_data: A dict or JSON string in the following format:
            {
                "1": {
                    "Sound_Noise_Sensor_MEMS": {
                        "timeseries_data": [
                            {"datetime": "2025-02-10 05:31:59", "reading_value": 87.5},
                            {"datetime": "2025-02-10 05:32:11", "reading_value": 92.0},
                            ...
                        ]
                    },
                    "Other_Sensor": { ... }
                },
                "2": {
                    "Sound_Noise_Sensor_MEMS": {
                        "timeseries_data": [
                            {"datetime": "2025-02-10 05:35:00", "reading_value": 89.0},
                            {"datetime": "2025-02-10 05:35:12", "reading_value": 91.5},
                            ...
                        ]
                    }
                }
            }
      - sensor_key: The sensor type key to look for (default: "Sound_Noise_Sensor_MEMS").
      - threshold: The noise level threshold (default: 90).

    Returns:
      A dictionary with summary statistics and an alert message.
    """
    # Parse sensor_data from JSON string if needed.
    if isinstance(sensor_data, str):
        try:
            sensor_data = json.loads(sensor_data)
        except Exception as e:
            logging.error(f"Error parsing sensor_data JSON: {e}")
            return {"error": "Invalid sensor_data JSON"}

    # Aggregate readings for the specified sensor_key across all sensor IDs.
    aggregated_readings = []
    for sensor_id, sensor_types in sensor_data.items():
        if sensor_key in sensor_types:
            readings = sensor_types[sensor_key].get("timeseries_data", [])
            aggregated_readings.extend(readings)

    if not aggregated_readings:
        return {"error": f"No data found for {sensor_key}"}

    try:
        df = pd.DataFrame(aggregated_readings)
        if "datetime" in df.columns:
            df = df.rename(columns={"datetime": "timestamp"})
        df["timestamp"] = pd.to_datetime(df["timestamp"])
        df = df.sort_values(by="timestamp")
        latest = df.iloc[-1]["reading_value"]
        summary = {
            "mean": float(df["reading_value"].mean()),
            "min": float(df["reading_value"].min()),
            "max": float(df["reading_value"].max()),
            "std": float(df["reading_value"].std()),
            "latest": float(latest),
        }
        summary["alert"] = (
            "High noise level" if latest > threshold else "Normal noise level"
        )
        return summary
    except Exception as e:
        logging.error(f"Error analyzing noise levels: {e}")
        return {"error": "Failed to analyze noise levels"}

In [123]:
alerts = analyze_noise_levels(json_data)
print("\nnoise_levels analysis:")
print(json.dumps(alerts, indent=2))


noise_levels analysis:
{
  "mean": 139.06666666666666,
  "min": 136.0,
  "max": 147.0,
  "std": 1.3157546089916623,
  "latest": 140.0,
  "alert": "High noise level"
}


In [ ]:
def analyze_air_quality(
    sensor_data, sensor_key="Air_Quality_Sensor", thresholds=(50, 100)
):
    """
    Analyzes air quality sensor data from a nested JSON structure.

    Aggregates readings for the specified sensor_key across sensor IDs.
    Computes the average air quality index and classifies it based on thresholds.

    Expected input (as a Python dict or JSON string):
      {
         "1": {
             "Air_Quality_Sensor": {
                 "timeseries_data": [
                     {"datetime": "2025-02-10 05:31:59", "reading_value": 45},
                     {"datetime": "2025-02-10 05:32:11", "reading_value": 50},
                     ...
                 ]
             },
             "Other_Sensor": { ... }
         },
         "2": {
             "Air_Quality_Sensor": {
                 "timeseries_data": [
                     {"datetime": "2025-02-10 05:35:00", "reading_value": 55},
                     {"datetime": "2025-02-10 05:35:12", "reading_value": 60},
                     ...
                 ]
             }
         }
      }

    Returns a dictionary containing:
        - average_air_quality: computed average reading_value.
        - status: classification ("Good", "Moderate", or "Poor").
        - min: minimum reading_value.
        - max: maximum reading_value.
    """
    # Parse sensor_data if it's a JSON string.
    if isinstance(sensor_data, str):
        try:
            sensor_data = json.loads(sensor_data)
        except Exception as e:
            logging.error(f"Error parsing sensor_data JSON: {e}")
            return {"error": "Invalid sensor_data JSON"}

    # Aggregate readings for the specified sensor_key across sensor IDs.
    aggregated_readings = []
    for sensor_id, sensor_types in sensor_data.items():
        if sensor_key in sensor_types:
            readings = sensor_types[sensor_key].get("timeseries_data", [])
            aggregated_readings.extend(readings)

    if not aggregated_readings:
        return {"error": f"No data found for {sensor_key}"}

    try:
        df = pd.DataFrame(aggregated_readings)
        if "datetime" in df.columns:
            df = df.rename(columns={"datetime": "timestamp"})
        df["timestamp"] = pd.to_datetime(df["timestamp"])
        df = df.sort_values(by="timestamp")
        avg_quality = float(df["reading_value"].mean())
        if avg_quality <= thresholds[0]:
            status = "Good"
        elif avg_quality <= thresholds[1]:
            status = "Moderate"
        else:
            status = "Poor"
        return {
            "average_air_quality": avg_quality,
            "status": status,
            "min": float(df["reading_value"].min()),
            "max": float(df["reading_value"].max()),
        }
    except Exception as e:
        logging.error(f"Error analyzing air quality: {e}")
        return {"error": "Failed to analyze air quality"}

In [128]:
print("\nAir Quality Analysis:")
print(json.dumps(analyze_air_quality(json_data), indent=2))


Air Quality Analysis:
{
  "average_air_quality": 10.041269841269841,
  "status": "Good",
  "min": 8.0,
  "max": 12.0
}


In [ ]:
def analyze_formaldehyde_levels(
    sensor_data, sensor_key="Formaldehyde_Level_Sensor", threshold=0.1
):
    """
    Analyzes formaldehyde sensor readings from a nested JSON structure.

    Aggregates readings for the specified sensor_key across sensor IDs.
    Computes summary statistics and flags if the latest reading exceeds the threshold.

    Expected input (as a Python dict or JSON string):
      {
         "1": {
             "Formaldehyde_Level_Sensor": {
                 "timeseries_data": [
                     {"datetime": "2025-02-10 05:31:59", "reading_value": 0.08},
                     {"datetime": "2025-02-10 05:32:11", "reading_value": 0.09},
                     ...
                 ]
             },
             "Other_Sensor": { ... }
         },
         "2": {
             "Formaldehyde_Level_Sensor": {
                 "timeseries_data": [
                     {"datetime": "2025-02-10 05:35:00", "reading_value": 0.11},
                     {"datetime": "2025-02-10 05:35:12", "reading_value": 0.10},
                     ...
                 ]
             }
         }
      }

    Returns a dictionary containing:
      - mean, min, max, std, and latest reading_value.
      - an alert message if the latest reading exceeds the threshold.
    """
    # Parse sensor_data if it is a JSON string.
    if isinstance(sensor_data, str):
        try:
            sensor_data = json.loads(sensor_data)
        except Exception as e:
            logging.error(f"Error parsing sensor_data JSON: {e}")
            return {"error": "Invalid sensor_data JSON"}

    # Aggregate readings for the specified sensor_key across all sensor IDs.
    aggregated_readings = []
    for sensor_id, sensor_types in sensor_data.items():
        if sensor_key in sensor_types:
            readings = sensor_types[sensor_key].get("timeseries_data", [])
            aggregated_readings.extend(readings)

    if not aggregated_readings:
        return {"error": f"No data found for {sensor_key}"}

    try:
        df = pd.DataFrame(aggregated_readings)
        if "datetime" in df.columns:
            df = df.rename(columns={"datetime": "timestamp"})
        df["timestamp"] = pd.to_datetime(df["timestamp"])
        df = df.sort_values(by="timestamp")
        latest = float(df.iloc[-1]["reading_value"])
        summary = {
            "mean": float(df["reading_value"].mean()),
            "min": float(df["reading_value"].min()),
            "max": float(df["reading_value"].max()),
            "std": float(df["reading_value"].std()),
            "latest": latest,
        }
        summary["alert"] = (
            "High formaldehyde level"
            if latest > threshold
            else "Normal formaldehyde level"
        )
        return summary
    except Exception as e:
        logging.error(f"Error analyzing formaldehyde levels: {e}")
        return {"error": "Failed to analyze formaldehyde levels"}

In [131]:
print("\nFormaldehyde Levels Analysis:")
print(json.dumps(analyze_formaldehyde_levels(json_data), indent=2))


Formaldehyde Levels Analysis:
{
  "mean": 0.10247619047619048,
  "min": 0.07,
  "max": 0.16,
  "std": 0.022830626111125896,
  "latest": 0.07,
  "alert": "Normal formaldehyde level"
}


In [132]:
def analyze_co2_levels(sensor_data, sensor_key="CO2_Level_Sensor", threshold=1000):
    """
    Analyzes CO2 sensor readings from a nested JSON structure.

    Aggregates readings for the specified sensor_key across sensor IDs,
    computes summary statistics, and flags if the latest reading exceeds the threshold.

    Expected input (as a Python dict or JSON string):
      {
          "1": {
              "CO2_Level_Sensor": {
                  "timeseries_data": [
                      {"datetime": "2025-02-10 05:31:59", "reading_value": 950},
                      {"datetime": "2025-02-10 05:32:11", "reading_value": 980},
                      ...
                  ]
              },
              "Other_Sensor": { ... }
          },
          "2": {
              "CO2_Level_Sensor": {
                  "timeseries_data": [
                      {"datetime": "2025-02-10 05:35:00", "reading_value": 1020},
                      {"datetime": "2025-02-10 05:35:12", "reading_value": 1005},
                      ...
                  ]
              }
          }
      }

    Returns a dictionary with:
      - mean, min, max, std, and latest reading_value.
      - an alert message if the latest reading exceeds the threshold.
    """
    # Parse sensor_data if provided as a JSON string.
    if isinstance(sensor_data, str):
        try:
            sensor_data = json.loads(sensor_data)
        except Exception as e:
            logging.error(f"Error parsing sensor_data JSON: {e}")
            return {"error": "Invalid sensor_data JSON"}

    # Aggregate readings for the specified sensor_key across sensor IDs.
    aggregated_readings = []
    for sensor_id, sensor_types in sensor_data.items():
        if sensor_key in sensor_types:
            readings = sensor_types[sensor_key].get("timeseries_data", [])
            aggregated_readings.extend(readings)

    if not aggregated_readings:
        return {"error": f"No data found for {sensor_key}"}

    try:
        df = pd.DataFrame(aggregated_readings)
        if "datetime" in df.columns:
            df = df.rename(columns={"datetime": "timestamp"})
        df["timestamp"] = pd.to_datetime(df["timestamp"])
        df = df.sort_values(by="timestamp")
        latest = float(df.iloc[-1]["reading_value"])
        summary = {
            "mean": float(df["reading_value"].mean()),
            "min": float(df["reading_value"].min()),
            "max": float(df["reading_value"].max()),
            "std": float(df["reading_value"].std()),
            "latest": latest,
        }
        summary["alert"] = (
            "High CO2 level" if latest > threshold else "Normal CO2 level"
        )
        return summary
    except Exception as e:
        logging.error(f"Error analyzing CO2 levels: {e}")
        return {"error": "Failed to analyze CO2 levels"}

In [133]:
print("\nCO2 Levels Analysis:")
print(json.dumps(analyze_co2_levels(json_data), indent=2))


CO2 Levels Analysis:
{
  "mean": 513.6984126984127,
  "min": 495.0,
  "max": 528.0,
  "std": 7.023697196862493,
  "latest": 519.0,
  "alert": "Normal CO2 level"
}


In [ ]:
def analyze_pm_levels(
    sensor_data,
    sensor_keys=[
        "PM1_Level_Sensor_Standard",
        "PM2_5_Level_Sensor_Standard",
        "PM10_Level_Sensor_Standard",
    ],
    thresholds={
        "PM1_Level_Sensor_Standard": 50,
        "PM2_5_Level_Sensor_Standard": 30,
        "PM10_Level_Sensor_Standard": 50,
    },
):
    """
    Analyzes particulate matter (PM) sensor data from a nested JSON structure.

    Expected input (as a Python dict or JSON string):
      {
          "1": {
              "PM1_Level_Sensor_Standard": {
                  "timeseries_data": [ ... ]
              },
              "PM2.5_Level_Sensor_Standard": {
                  "timeseries_data": [ ... ]
              },
              "PM10_Level_Sensor_Standard": {
                  "timeseries_data": [ ... ]
              },
              "Other_Sensor": { ... }
          },
          "2": {
              "PM1_Level_Sensor_Standard": { "timeseries_data": [ ... ] },
              "PM2.5_Level_Sensor_Standard": { "timeseries_data": [ ... ] },
              "PM10_Level_Sensor_Standard": { "timeseries_data": [ ... ] }
          }
      }

    For each sensor key, the function aggregates readings across sensor IDs,
    computes summary statistics (mean, min, max, std, latest value) and flags if
    the latest reading exceeds its defined threshold.

    Returns a dictionary mapping each sensor key to its analysis summary.
    """
    # Parse sensor_data if it's a JSON string.
    if isinstance(sensor_data, str):
        try:
            sensor_data = json.loads(sensor_data)
        except Exception as e:
            logging.error(f"Error parsing sensor_data JSON: {e}")
            return {"error": "Invalid sensor data"}

    # If sensor_data is a list (data for a single sensor), wrap it in a dict.
    if isinstance(sensor_data, list):
        sensor_data = {"1": sensor_data}

    analysis = {}
    for key in sensor_keys:
        # Aggregate readings for this key across all sensor IDs.
        aggregated_readings = []
        for sensor_id, sensor_types in sensor_data.items():
            if key in sensor_types:
                readings = sensor_types[key].get("timeseries_data", [])
                aggregated_readings.extend(readings)
        if not aggregated_readings:
            analysis[key] = {"error": "No data available"}
            continue
        try:
            df = pd.DataFrame(aggregated_readings)
            if "datetime" in df.columns:
                df = df.rename(columns={"datetime": "timestamp"})
            df["timestamp"] = pd.to_datetime(df["timestamp"])
            df = df.sort_values(by="timestamp")
            latest = float(df.iloc[-1]["reading_value"])
            summary = {
                "mean": float(df["reading_value"].mean()),
                "min": float(df["reading_value"].min()),
                "max": float(df["reading_value"].max()),
                "std": float(df["reading_value"].std()),
                "latest": latest,
            }
            thres = thresholds.get(key, None)
            if thres is not None:
                summary["alert"] = (
                    f"High {key} reading" if latest > thres else f"Normal {key} reading"
                )
            else:
                summary["alert"] = "Threshold not defined"
            analysis[key] = summary
        except Exception as e:
            logging.error(f"Error analyzing {key}: {e}")
            analysis[key] = {"error": f"Failed to analyze {key}"}
    return analysis

In [135]:
print("\nPM Levels Analysis:")
print(json.dumps(analyze_pm_levels(json_data), indent=2))


PM Levels Analysis:
{
  "PM1_Level_Sensor_Standard": {
    "mean": 4.203174603174603,
    "min": 4.0,
    "max": 5.0,
    "std": 0.4030015781165281,
    "latest": 4.0,
    "alert": "Normal PM1_Level_Sensor_Standard reading"
  },
  "PM2_5_Level_Sensor_Standard": {
    "mean": 5.8126984126984125,
    "min": 5.0,
    "max": 7.0,
    "std": 0.7485638188698672,
    "latest": 6.0,
    "alert": "Normal PM2_5_Level_Sensor_Standard reading"
  },
  "PM10_Level_Sensor_Standard": {
    "mean": 5.8126984126984125,
    "min": 5.0,
    "max": 7.0,
    "std": 0.7485638188698672,
    "latest": 6.0,
    "alert": "Normal PM10_Level_Sensor_Standard reading"
  }
}


In [ ]:
def analyze_temperatures(
    sensor_data, sensor_key="Air_Temperature_Sensor", acceptable_range=(18, 27)
):
    """
    Analyzes temperature sensor data from a nested JSON structure.

    Aggregates readings for the specified sensor_key across sensor IDs.
    Computes summary statistics (mean, min, max, std, and latest reading) and flags if the latest
    reading is outside the acceptable range.

    Expected input (as a Python dict or JSON string):
      {
         "1": {
             "Air_Temperature_Sensor": {
                 "timeseries_data": [
                     {"datetime": "2025-02-10 05:31:59", "reading_value": 22.5},
                     {"datetime": "2025-02-10 05:32:11", "reading_value": 23.0},
                     ...
                 ]
             },
             "Other_Sensor": { ... }
         },
         "2": {
             "Air_Temperature_Sensor": {
                 "timeseries_data": [
                     {"datetime": "2025-02-10 05:33:00", "reading_value": 24.0},
                     {"datetime": "2025-02-10 05:33:15", "reading_value": 23.5},
                     ...
                 ]
             }
         }
      }

    Returns:
      A dictionary with the computed summary statistics and an alert message.
    """
    # Parse sensor_data if provided as a JSON string.
    if isinstance(sensor_data, str):
        try:
            sensor_data = json.loads(sensor_data)
        except Exception as e:
            logging.error(f"Error parsing sensor_data JSON: {e}")
            return {"error": "Invalid sensor_data JSON"}

    # Aggregate readings for the specified sensor_key across all sensor IDs.
    aggregated_readings = []
    for sensor_id, sensor_types in sensor_data.items():
        if sensor_key in sensor_types:
            readings = sensor_types[sensor_key].get("timeseries_data", [])
            aggregated_readings.extend(readings)

    if not aggregated_readings:
        return {"error": f"No data found for {sensor_key}"}

    try:
        df = pd.DataFrame(aggregated_readings)
        if "datetime" in df.columns:
            df = df.rename(columns={"datetime": "timestamp"})
        df["timestamp"] = pd.to_datetime(df["timestamp"])
        df = df.sort_values(by="timestamp")
        latest = float(df.iloc[-1]["reading_value"])
        summary = {
            "mean": float(df["reading_value"].mean()),
            "min": float(df["reading_value"].min()),
            "max": float(df["reading_value"].max()),
            "std": float(df["reading_value"].std()),
            "latest": latest,
        }
        summary["alert"] = (
            "Temperature out of range"
            if (latest < acceptable_range[0] or latest > acceptable_range[1])
            else "Temperature normal"
        )
        return summary
    except Exception as e:
        logging.error(f"Error analyzing temperatures: {e}")
        return {"error": "Failed to analyze temperatures"}

In [137]:
print("\nTemperature Analysis:")
print(json.dumps(analyze_temperatures(json_data), indent=2))


Temperature Analysis:
{
  "mean": 29.014,
  "min": 28.96,
  "max": 29.07,
  "std": 0.02495601225698345,
  "latest": 28.97,
  "alert": "Temperature out of range"
}


In [ ]:
def analyze_humidity(
    sensor_data, sensor_key="Zone_Air_Humidity_Sensor", acceptable_range=(30, 60)
):
    """
    Analyzes humidity sensor data from a nested JSON structure.

    Aggregates readings for the specified sensor_key across sensor IDs,
    computes summary statistics (mean, min, max, std, latest reading), and
    flags an alert if the latest reading is outside the acceptable range.

    Expected input (as a Python dict or JSON string):
      {
         "1": {
             "Zone_Air_Humidity_Sensor": {
                 "timeseries_data": [
                     {"datetime": "2025-02-10 05:31:59", "reading_value": 45},
                     {"datetime": "2025-02-10 05:32:11", "reading_value": 50},
                     ...
                 ]
             },
             "Other_Sensor": { ... }
         },
         "2": {
             "Zone_Air_Humidity_Sensor": {
                 "timeseries_data": [
                     {"datetime": "2025-02-10 05:35:00", "reading_value": 55},
                     {"datetime": "2025-02-10 05:35:12", "reading_value": 60},
                     ...
                 ]
             }
         }
      }

    Returns:
      A dictionary containing summary statistics and an alert message.
    """
    # Parse sensor_data if provided as a JSON string.
    if isinstance(sensor_data, str):
        try:
            sensor_data = json.loads(sensor_data)
        except Exception as e:
            logging.error(f"Error parsing sensor_data JSON: {e}")
            return {"error": "Invalid sensor_data JSON"}

    # Aggregate readings for the specified sensor_key across sensor IDs.
    aggregated_readings = []
    for sensor_id, sensor_types in sensor_data.items():
        if sensor_key in sensor_types:
            readings = sensor_types[sensor_key].get("timeseries_data", [])
            aggregated_readings.extend(readings)

    if not aggregated_readings:
        return {"error": f"No data found for {sensor_key}"}

    try:
        df = pd.DataFrame(aggregated_readings)
        if "datetime" in df.columns:
            df = df.rename(columns={"datetime": "timestamp"})
        df["timestamp"] = pd.to_datetime(df["timestamp"])
        df = df.sort_values(by="timestamp")
        latest = float(df.iloc[-1]["reading_value"])
        summary = {
            "mean": float(df["reading_value"].mean()),
            "min": float(df["reading_value"].min()),
            "max": float(df["reading_value"].max()),
            "std": float(df["reading_value"].std()),
            "latest": latest,
        }
        summary["alert"] = (
            "Humidity out of range"
            if (latest < acceptable_range[0] or latest > acceptable_range[1])
            else "Humidity normal"
        )
        return summary
    except Exception as e:
        logging.error(f"Error analyzing humidity: {e}")
        return {"error": "Failed to analyze humidity"}

In [139]:
print("\nHumidity Analysis:")
print(json.dumps(analyze_humidity(json_data), indent=2))


Humidity Analysis:
{
  "mean": 23.29187301587302,
  "min": 23.14,
  "max": 23.48,
  "std": 0.07501156854802313,
  "latest": 23.2,
  "alert": "Humidity out of range"
}


In [ ]:
def analyze_temperature_humidity(
    sensor_data,
    temp_key="Air_Temperature_Sensor",
    humidity_key="Zone_Air_Humidity_Sensor",
    temp_range=(18, 27),
    humidity_range=(30, 60),
):
    """
    Analyzes temperature and humidity sensor data from a nested JSON structure.

    Aggregates readings for the specified sensor keys across sensor IDs,
    computes individual summaries for temperature and humidity, and calculates
    a combined comfort index. The comfort index is computed by measuring how
    close the latest sensor readings are to the midpoints of the acceptable ranges.

    Parameters:
      - sensor_data: A dict or JSON string in the nested format.
      - temp_key: Sensor key for temperature (default: "Air_Temperature_Sensor").
      - humidity_key: Sensor key for humidity (default: "Zone_Air_Humidity_Sensor").
      - temp_range: Acceptable range for temperature (default: (18, 27)).
      - humidity_range: Acceptable range for humidity (default: (30, 60)).

    Returns:
      A dictionary with:
        - 'temperature': Summary from analyze_temperatures.
        - 'humidity': Summary from analyze_humidity.
        - 'comfort_index': A value between 0 and 100.
        - 'comfort_assessment': A qualitative assessment ("Comfortable", "Less comfortable", or "Uncomfortable").
    """
    # Parse sensor_data if it's a JSON string.
    if isinstance(sensor_data, str):
        try:
            sensor_data = json.loads(sensor_data)
        except Exception as e:
            logging.error(f"Error parsing sensor_data JSON: {e}")
            return {"error": "Invalid sensor data"}

    # Call the updated analysis functions (which expect the nested JSON structure)
    temp_summary = analyze_temperatures(
        sensor_data, sensor_key=temp_key, acceptable_range=temp_range
    )
    humidity_summary = analyze_humidity(
        sensor_data, sensor_key=humidity_key, acceptable_range=humidity_range
    )

    # Compute midpoints for the acceptable ranges.
    temp_mid = (temp_range[0] + temp_range[1]) / 2.0
    humidity_mid = (humidity_range[0] + humidity_range[1]) / 2.0

    try:
        temp_latest = temp_summary.get("latest", temp_mid)
        humidity_latest = humidity_summary.get("latest", humidity_mid)
        # Calculate deviations from the midpoints.
        temp_diff = abs(temp_latest - temp_mid)
        humidity_diff = abs(humidity_latest - humidity_mid)
        # Compute a simple comfort index: 100 minus weighted deviations.
        comfort_index = 100 - (temp_diff * 2 + humidity_diff * 1.5)
        # Ensure the index is within 0 to 100.
        comfort_index = max(0, min(100, comfort_index))
        comfort_index = float(comfort_index)
    except Exception as e:
        logging.error(f"Error computing comfort index: {e}")
        comfort_index = None

    combined = {
        "temperature": temp_summary,
        "humidity": humidity_summary,
        "comfort_index": comfort_index,
        "comfort_assessment": (
            "Comfortable"
            if comfort_index is not None and comfort_index > 70
            else (
                "Less comfortable"
                if comfort_index is not None and comfort_index > 40
                else "Uncomfortable"
            )
        ),
    }
    return combined

In [141]:
print("\nTemperature & Humidity Combined Analysis:")
print(json.dumps(analyze_temperature_humidity(json_data), indent=2))


Temperature & Humidity Combined Analysis:
{
  "temperature": {
    "mean": 29.014,
    "min": 28.96,
    "max": 29.07,
    "std": 0.02495601225698345,
    "latest": 28.97,
    "alert": "Temperature out of range"
  },
  "humidity": {
    "mean": 23.29187301587302,
    "min": 23.14,
    "max": 23.48,
    "std": 0.07501156854802313,
    "latest": 23.2,
    "alert": "Humidity out of range"
  },
  "comfort_index": 54.36,
  "comfort_assessment": "Less comfortable"
}


In [ ]:
def detect_potential_failures(sensor_data, time_window_hours=24, anomaly_threshold=3):
    """
    Detects potential sensor failures based on anomaly detection within a specified time window,
    using a nested JSON structure.

    Args:
      - sensor_data (dict or JSON string): Nested dictionary containing sensor time series data.
          Expected format:
          {
              "1": {
                  "Sensor_Type_A": {
                      "timeseries_data": [
                          {"datetime": "2025-02-10 05:31:59", "reading_value": 27.99},
                          {"datetime": "2025-02-10 05:32:11", "reading_value": 28.1},
                          ...
                      ]
                  },
                  "Sensor_Type_B": { ... }
              },
              "2": {
                  "Sensor_Type_A": { ... },
                  ...
              }
          }
      - time_window_hours (int): Time window in hours to analyze for potential failures.
      - anomaly_threshold (float): Z-score threshold for anomaly detection.

    Returns:
      - List of flattened sensor identifiers (e.g. "1_Sensor_Type_A") showing potential failures.
    """
    # Parse sensor_data from JSON string if necessary.
    if isinstance(sensor_data, str):
        try:
            sensor_data = json.loads(sensor_data)
        except Exception as e:
            logging.error(f"Error parsing sensor_data JSON: {e}")
            return []

    sensors_with_failures = []

    # Iterate over each sensor ID.
    for sensor_id, sensor_types in sensor_data.items():
        # Iterate over each sensor type within this sensor ID.
        for sensor_type, sensor_info in sensor_types.items():
            data_points = sensor_info.get("timeseries_data", [])
            # If there is no data, skip this sensor type.
            if not data_points:
                continue
            try:
                df = pd.DataFrame(data_points)
                if "datetime" in df.columns:
                    df = df.rename(columns={"datetime": "timestamp"})
                df["timestamp"] = pd.to_datetime(df["timestamp"])
                df = df.sort_values(by="timestamp")

                # Calculate rolling mean and standard deviation for anomaly detection.
                df["rolling_mean"] = (
                    df["reading_value"].rolling(window=5, min_periods=1).mean()
                )
                df["rolling_std"] = (
                    df["reading_value"].rolling(window=5, min_periods=1).std()
                )

                # Replace zeros in rolling_std with 1 to avoid division by zero.
                std_series = df["rolling_std"].replace(0, 1)
                df["zscore"] = np.abs(
                    (df["reading_value"] - df["rolling_mean"]) / std_series
                )

                # Identify potential failures where z-score exceeds the threshold.
                potential_failures = df[df["zscore"] > anomaly_threshold]

                if not potential_failures.empty:
                    latest_timestamp = df.iloc[-1]["timestamp"]
                    failures_in_window = potential_failures[
                        potential_failures["timestamp"]
                        >= (latest_timestamp - pd.Timedelta(hours=time_window_hours))
                    ]
                    if not failures_in_window.empty:
                        sensors_with_failures.append(f"{sensor_id}_{sensor_type}")
            except Exception as e:
                logging.error(
                    f"Error processing sensor {sensor_id} ({sensor_type}): {e}"
                )

    return sensors_with_failures

In [164]:
potential_failures = detect_potential_failures(json_data)
if potential_failures:
    print(f"Sensors showing potential failures:", potential_failures)
else:
    print("No sensors showing potential failures within the next 24 hours.")

No sensors showing potential failures within the next 24 hours.


In [ ]:
def forecast_downtimes(sensor_data):
    """
    Forecast potential downtimes using predictive analytics from a nested JSON structure.

    Args:
      - sensor_data (dict or JSON string): Nested sensor data, where each outer key is a sensor ID
        and each inner key is a sensor type containing a "timeseries_data" list.
        Example:
          {
              "1": {
                  "Sensor_Type_A": {
                      "timeseries_data": [
                          {"datetime": "2025-02-10 05:31:59", "reading_value": 27.99},
                          {"datetime": "2025-02-10 05:32:11", "reading_value": 28.1},
                          ...
                      ]
                  },
                  "Sensor_Type_B": { ... }
              },
              "2": {
                  "Sensor_Type_A": { ... },
                  ...
              }
          }
      - The function forecasts downtimes based on rolling statistics.

    Returns:
      - A dictionary mapping each flattened sensor identifier (e.g. "1_Sensor_Type_A") to a list of timestamps
        (as strings) forecasted for potential downtimes.
    """
    # Parse sensor_data if it is a JSON string.
    if isinstance(sensor_data, str):
        try:
            sensor_data = json.loads(sensor_data)
        except Exception as e:
            logging.error(f"Error parsing sensor_data JSON: {e}")
            return {}

    downtimes_forecast = {}

    # Iterate over each sensor ID.
    for sensor_id, sensor_types in sensor_data.items():
        # Iterate over each sensor type for this sensor ID.
        for sensor_type, sensor_info in sensor_types.items():
            timeseries_data = sensor_info.get("timeseries_data", [])
            unique_key = f"{sensor_id}_{sensor_type}"

            if not timeseries_data:
                downtimes_forecast[unique_key] = []
                continue

            try:
                df = pd.DataFrame(timeseries_data)
                # Rename "datetime" column to "timestamp" if present.
                if "datetime" in df.columns:
                    df = df.rename(columns={"datetime": "timestamp"})
                df["timestamp"] = pd.to_datetime(df["timestamp"])
                df = df.sort_values(by="timestamp")
                df = df.set_index("timestamp")

                # Calculate rolling mean and standard deviation.
                df["rolling_mean"] = (
                    df["reading_value"].rolling(window=5, min_periods=1).mean()
                )
                df["rolling_std"] = (
                    df["reading_value"].rolling(window=5, min_periods=1).std()
                )

                # Define a threshold series: rolling_mean - 2 * rolling_std.
                threshold_series = df["rolling_mean"] - 2 * df["rolling_std"]
                potential_downtimes = df[df["reading_value"] < threshold_series]

                # Extract timestamps of potential downtimes.
                forecasted = potential_downtimes.index.strftime(
                    "%Y-%m-%d %H:%M:%S"
                ).tolist()
                downtimes_forecast[unique_key] = forecasted
            except Exception as e:
                logging.error(
                    f"Error forecasting downtimes for sensor {unique_key}: {e}"
                )
                downtimes_forecast[unique_key] = []

    return downtimes_forecast

In [163]:
forecast = forecast_downtimes(json_data)
print("Forecasted Downtimes:")
for sensor, downtimes in forecast.items():
    print(f"{sensor}: {downtimes}")

Forecasted Downtimes:
2_Air_Temperature_Sensor: []
2_Zone_Air_Humidity_Sensor: []
2_PM2_5_Level_Sensor_Standard: []
2_PM10_Level_Sensor_Standard: []
2_NO2_Level_Sensor: []
2_CO_Level_Sensor: []
2_CO2_Level_Sensor: []
2_Sound_Noise_Sensor_MEMS: []
2_Air_Quality_Sensor: []
2_Formaldehyde_Level_Sensor: []
2_PM1_Level_Sensor_Standard: []


In [165]:
print(json_data)

{
    "2": {
        "Air_Temperature_Sensor": {
            "timeseries_data": [
                {
                    "datetime": "2025-02-10 00:00:00",
                    "reading_value": 29.06
                },
                {
                    "datetime": "2025-02-10 00:00:11",
                    "reading_value": 29.05
                },
                {
                    "datetime": "2025-02-10 00:00:23",
                    "reading_value": 29.07
                },
                {
                    "datetime": "2025-02-10 00:00:34",
                    "reading_value": 29.06
                },
                {
                    "datetime": "2025-02-10 00:00:45",
                    "reading_value": 29.06
                },
                {
                    "datetime": "2025-02-10 00:00:57",
                    "reading_value": 29.04
                },
                {
                    "datetime": "2025-02-10 00:01:08",
                    "reading_value"

In [16]:
print(json_data)

{
    "2": {
        "Air_Temperature_Sensor": {
            "timeseries_data": [
                {
                    "datetime": "2025-02-10 00:00:00",
                    "reading_value": 29.06
                },
                {
                    "datetime": "2025-02-10 00:00:11",
                    "reading_value": 29.05
                },
                {
                    "datetime": "2025-02-10 00:00:23",
                    "reading_value": 29.07
                },
                {
                    "datetime": "2025-02-10 00:00:34",
                    "reading_value": 29.06
                },
                {
                    "datetime": "2025-02-10 00:00:45",
                    "reading_value": 29.06
                },
                {
                    "datetime": "2025-02-10 00:00:57",
                    "reading_value": 29.04
                },
                {
                    "datetime": "2025-02-10 00:01:08",
                    "reading_value"

In [63]:
import requests
import json

# Use the SQL query results as the sensor data payload.
# Ensure that `json_data` is defined and properly formatted.
analytics_payload = {
    "analysis_type": "detect_potential_failures",  # Choose the analysis function.
    "sensor_data": json_data,
}

analytics_url = "http://localhost:6000/analytics/run"
try:
    r_analytics = requests.post(analytics_url, json=analytics_payload)
    r_analytics.raise_for_status()  # Raises HTTPError for bad responses.
    analytics_result = r_analytics.json()
    print("Analytics Endpoint Response:")
    print(json.dumps(analytics_result, indent=2))
except Exception as e:
    print("Error calling analytics endpoint:", e)

Analytics Endpoint Response:
[]


In [ ]:
sparql_query = "SELECT ?tag WHERE { brick:Supply_Air_Flow_Sensor brick:hasAssociatedTag ?tag . }"  # test

SELECT ?tag WHERE { brick:Supply_Air_Flow_Sensor brick:hasAssociatedTag ?tag . }


In [14]:
FUSEKI_URL = "http://localhost:3030/abacws-sensor-network/sparql"  # Apache Jena Fuseki endpoint local

In [4]:
from typing import Any, Text, Dict, List, Tuple

In [10]:
def execute_sparql_query(self, sparql_query: str) -> Dict:
    """
    Executes the given SPARQL query against the Fuseki endpoint.
    """
    sparql = SPARQLWrapper(FUSEKI_URL)
    sparql.setQuery(sparql_query)
    sparql.setReturnFormat(JSON)
    try:
        results = sparql.query().convert()
        logging.info("SPARQL query executed successfully.")
        return results
    except Exception as e:
        logging.error(f"Error executing SPARQL query: {e}")
        return None

In [11]:
def add_sparql_prefixes(self, sparql_query: str) -> str:
    """
    Appends a set of predefined SPARQL prefixes to the query.
    """
    prefixes = [
        "PREFIX brick: <https://brickschema.org/schema/Brick#>",
        "PREFIX dcterms: <http://purl.org/dc/terms/>",
        "PREFIX owl: <http://www.w3.org/2002/07/owl#>",
        "PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>",
        "PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>",
        "PREFIX sh: <http://www.w3.org/ns/shacl#>",
        "PREFIX skos: <http://www.w3.org/2004/02/skos/core#>",
        "PREFIX sosa: <http://www.w3.org/ns/sosa/>",
        "PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>",
        "PREFIX tag: <https://brickschema.org/schema/BrickTag#>",
        "PREFIX bldg: <http://buildsys.org/ontologies/bldg1#>",
        "PREFIX bsh: <https://brickschema.org/schema/BrickShape#>",
        "PREFIX s223: <http://data.ashrae.org/standard223#>",
        "PREFIX bacnet: <http://data.ashrae.org/bacnet/2020#>",
        "PREFIX g36: <http://data.ashrae.org/standard223/1.0/extensions/g36#>",
        "PREFIX qkdv: <http://qudt.org/vocab/dimensionvector/>",
        "PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>",
        "PREFIX qudt: <http://qudt.org/schema/qudt/>",
        "PREFIX rec: <https://w3id.org/rec#>",
        "PREFIX ref: <https://brickschema.org/schema/Brick/ref#>",
        "PREFIX s223tobrick: <https://brickschema.org/extension/brick_extension_interpret_223#>",
        "PREFIX schema1: <http://schema.org/>",
        "PREFIX unit: <http://qudt.org/vocab/unit/>",
        "PREFIX vcard: <http://www.w3.org/2006/vcard/ns#>",
    ]
    return "\n".join(prefixes) + "\n" + sparql_query

In [16]:
full_sparql_query = self.add_sparql_prefixes(sparql_query)
logging.info(f"Full SPARQL query: {full_sparql_query}")

NameError: name 'self' is not defined

In [19]:
from SPARQLWrapper import SPARQLWrapper, JSON
import logging

FUSEKI_URL = "http://localhost:3030/abacws-sensor-network/sparql"


def add_sparql_prefixes(sparql_query: str) -> str:
    prefixes = [
        "PREFIX brick: <https://brickschema.org/schema/Brick#>",
        "PREFIX dcterms: <http://purl.org/dc/terms/>",
        "PREFIX owl: <http://www.w3.org/2002/07/owl#>",
        "PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>",
        "PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>",
        "PREFIX sh: <http://www.w3.org/ns/shacl#>",
        "PREFIX skos: <http://www.w3.org/2004/02/skos/core#>",
        "PREFIX sosa: <http://www.w3.org/ns/sosa/>",
        "PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>",
        "PREFIX tag: <https://brickschema.org/schema/BrickTag#>",
        "PREFIX bldg: <http://abacwsbuilding.cardiff.ac.uk/abacws#> ",
        "PREFIX bsh: <https://brickschema.org/schema/BrickShape#>",
        "PREFIX s223: <http://data.ashrae.org/standard223#>",
        "PREFIX bacnet: <http://data.ashrae.org/bacnet/2020#>",
        "PREFIX g36: <http://data.ashrae.org/standard223/1.0/extensions/g36#>",
        "PREFIX qkdv: <http://qudt.org/vocab/dimensionvector/>",
        "PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>",
        "PREFIX qudt: <http://qudt.org/schema/qudt/>",
        "PREFIX rec: <https://w3id.org/rec#>",
        "PREFIX ref: <https://brickschema.org/schema/Brick/ref#>",
        "PREFIX s223tobrick: <https://brickschema.org/extension/brick_extension_interpret_223#>",
        "PREFIX schema1: <http://schema.org/>",
        "PREFIX unit: <http://qudt.org/vocab/unit/>",
        "PREFIX vcard: <http://www.w3.org/2006/vcard/ns#>",
    ]
    return "\n".join(prefixes) + "\n" + sparql_query


def execute_sparql_query(sparql_query: str) -> dict:
    sparql = SPARQLWrapper(FUSEKI_URL)
    sparql.setQuery(sparql_query)
    sparql.setReturnFormat(JSON)
    try:
        results = sparql.query().convert()
        logging.info("SPARQL query executed successfully.")
        return results
    except Exception as e:
        logging.error(f"Error executing SPARQL query: {e}")
        return None


# Now call the function:
sparql_query = "SELECT ?label WHERE { bldg:mqtwo5.26 rdfs:label ?label . }"
full_sparql_query = add_sparql_prefixes(sparql_query)
logging.info(f"Full SPARQL query: {full_sparql_query}")
results = execute_sparql_query(full_sparql_query)

print(results)

{'head': {'vars': ['label']}, 'results': {'bindings': [{'label': {'type': 'literal', 'value': 'Room MQ2 Gas Sensor'}}]}}


In [29]:
bindings = results.get("results", {}).get("bindings", [])
has_timeseries = any("hasTimeseriesId" in binding for binding in bindings)
print(has_timeseries)

False


## OpemnAI API

In [ ]:
# In your terminal, first run:
# pip install openai

import os
from openai import OpenAI

XAI_API_KEY = os.getenv("XAI_API_KEY")
client = OpenAI(
    api_key=XAI_API_KEY,
    base_url="https://api.x.ai/v1",
)

completion = client.chat.completions.create(
    model="grok-2-latest",
    messages=[
        {
            "role": "system",
            "content": "You are Grok, a chatbot inspired by the Hitchhikers Guide to the Galaxy.",
        },
        {
            "role": "user",
            "content": "What is the meaning of life, the universe, and everything?",
        },
    ],
)

print(completion.choices[0].message.content)

In [ ]:
import rdflib
import random

# Initialize RDF graph
g = rdflib.Graph()

# Simulated smart home ontology (replace with BrickSchema .ttl file if available)
ontology = """
@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix ex: <http://example.org/> .

ex:LivingRoom a brick:Room .
ex:Kitchen a brick:Room .
ex:Bedroom a brick:Room .

ex:TempSensor1 a brick:Temperature_Sensor ;
    brick:hasLocation ex:LivingRoom .
ex:TempSensor2 a brick:Temperature_Sensor ;
    brick:hasLocation ex:Kitchen .
ex:Light1 a brick:Lighting_System ;
    brick:hasLocation ex:Bedroom .
ex:Light2 a brick:Lighting_System ;
    brick:hasLocation ex:Kitchen .
"""

# Parse the ontology
g.parse(data=ontology, format="turtle")

# Templates for natural language questions and SPARQL queries
question_templates = [
    ("Which {class}s are in the {location}?", 
     "PREFIX brick: <https://brickschema.org/schema/Brick#> SELECT ?item WHERE {{ ?item a brick:{class} . ?item brick:hasLocation ex:{location} . }}"),
    ("What {class}s are located in {location}?", 
     "PREFIX brick: <https://brickschema.org/schema/Brick#> SELECT ?item WHERE {{ ?item a brick:{class} . ?item brick:hasLocation ex:{location} . }}"),
    ("Show me all {class}s in {location}.", 
     "PREFIX brick: <https://brickschema.org/schema/Brick#> SELECT ?item WHERE {{ ?item a brick:{class} . ?item brick:hasLocation ex:{location} . }}"),
]

# Classes and locations from ontology
classes = ["Temperature_Sensor", "Lighting_System"]
locations = ["LivingRoom", "Kitchen", "Bedroom"]

# Generate question-SPARQL pairs
def generate_pairs(num_pairs=100):
    pairs = []
    for _ in range(num_pairs):
        template = random.choice(question_templates)
        class_name = random.choice(classes)
        location = random.choice(locations)
        
        question = template[0].format(class=class_name.replace("_", " ").lower(), location=location)
        sparql = template[1].format(class=class_name, location=location)
        pairs.append((question, sparql))
    return pairs

# Generate and save data
training_pairs = generate_pairs(1000)  # Generate 1000 pairs
with open("sample_training_data_context.txt", "w") as f:
    for question, sparql in training_pairs:
        f.write(f"query: {question}\tsparql: {sparql}\n")

print("Training data generated and saved to 'training_data.txt'.")